# Fetching functions

In [1]:
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.functions import udf
from pyspark.sql import types as T
from pyspark.sql.window import Window

def basic_srp_processing(table, start_dt, end_dt, site_id, leaf_ids):
    df_srch = table \
        .where((F.col("session_start_dt") >= start_dt)
               & (F.col("session_start_dt") < end_dt)
               & (F.col("site_id") == site_id)
               & (F.col("user_id") != 0)
               & (F.col("dominant_category_id").isin(leaf_ids))) \
        .select('site_id',
                'session_skey',
                'user_id',
                'session_start_dt',
                'clean_aspects',
                'clean_query',
                'dominant_category_id')

    return df_srch


from datetime import date, timedelta

def delta_to_dates(time_delta: int) -> dict:
    today = date.today()
    today_min_delta = today - timedelta(days=time_delta)
    start_dt = str(today_min_delta)
    end_dt = str(today)
    print(f"Given time_delta: {time_delta}, updated start_dt: {start_dt} and end_dt: {end_dt}")

    return {'start_dt': start_dt, "end_dt": end_dt}

def filter_srp_by_col(df_srch, column):
    df_col_filter = df_srch.where(F.col(column) != '')

    if column == "clean_aspects":
        return df_col_filter.dropDuplicates(
            [column, 'clean_query', 'session_start_dt', 'user_id', 'dominant_category_id'])
    else:
        return df_col_filter.dropDuplicates(
            [column, 'session_skey', 'user_id', 'dominant_category_id'])
    
    
def filter_by_regex(df, primary_regex, secondary_regex, q_col="clean_query"):
    df_primary = df.filter(F.col(q_col).rlike(primary_regex))

    df_no_primary = df.filter(~F.col(q_col).rlike(primary_regex))
    df_no_primary = df_no_primary.filter(F.col(q_col).rlike(secondary_regex))

    df_final = df_primary.union(df_no_primary)

    return df_final

# Processing functions

In [2]:
from typing import List, Tuple, Optional
import pandas as pd
import re
import numpy as np
from collections import Counter

from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as F
from datetime import date, timedelta
import multiprocessing as mp

import datetime
import time


def print_time() -> str:
    return datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')


def delta_to_dates(time_delta: int) -> dict:
    today = date.today()
    today_min_delta = today - timedelta(days=time_delta)
    start_dt = str(today_min_delta)
    end_dt = str(today)
    print(f"Given time_delta: {time_delta}, updated start_dt: {start_dt} and end_dt: {end_dt}")

    return {'start_dt': start_dt, "end_dt": end_dt}


def load_spark_df_to_pandas(spark, hdfs_path):
    df = spark.read.parquet(hdfs_path)
    pdf = df.toPandas()
    print(f"DataFrame shape: {pdf.shape}")
    return pdf


def extract_query_size(query: str, primary_regex=r'(size|sz) (\d+(?:\.\d+)?)', secondary_regex=None):
    find_size = re.findall(primary_regex, query)
    if find_size:
        return find_size[0][1]
    else:
        if secondary_regex:
            find_size2 = re.findall(secondary_regex, query)
            if find_size2:
                return find_size2[0]

    return None


def size_query_per_user(queries: List, max_size: float, min_size: float, secondary_regex=None) -> Counter:
    out = []
    for q in queries:
        q_size = extract_query_size(q, secondary_regex=secondary_regex)
        if q_size:
            out.append(q_size)

    out_float = convert_to_float(out)

    out_rm_outliers = rm_sizes_outliers(out_float, max_size, min_size)

    return Counter(out_rm_outliers)


def convert_to_float(input_list: List) -> List:
    out = []
    for i in input_list:
        try:
            i = float(i)
            out.append(i)
        except ValueError:
            pass
            # print(f"{i} is not a number")
    return out


def rm_sizes_outliers(input_list: List, max_size: float, min_size: float) -> List:
    out = []
    for i in input_list:
        if max_size >= i >= min_size:
            out.append(i)
    return out


def calc_max_min_diff(sizes: List, default=None) -> List:
    out = []
    for i in sizes:
        if i:
            maxi = np.max(list(i.elements()))
            mini = np.min(list(i.elements()))
            out.append(maxi - mini)
        else:
            out.append(default)
    return out


def mode(x):
    values, counts = np.unique(x, return_counts=True)
    m = counts.argmax()
    return values[m]


def agg_sizes(input_df: pd.DataFrame, sizes_col: str, col_prefix: str) -> pd.DataFrame:
    df = input_df.copy()

    df[f'{col_prefix}_std'] = [np.std(list(i.elements())) for i in df[sizes_col]]

    df[f'{col_prefix}_mean'] = [np.mean(list(i.elements())) for i in df[sizes_col]]

    df[f'{col_prefix}_median'] = [np.median(list(i.elements())) for i in df[sizes_col]]

    df[f'{col_prefix}_mode'] = [mode(list(i.elements())) for i in df[sizes_col]]

    df[f'{col_prefix}_maxMinDiff'] = calc_max_min_diff(df[sizes_col])

    df[f'{col_prefix}_countOfEvidence'] = [sum(i.values()) for i in df[sizes_col]]

    return df


def size_filters_per_user(aspects: List[str], max_size: float, min_size: float, pattern=r'size') -> Counter:
    user_vals = []
    for a in aspects:
        for i in a.split(";"):
            size_search = re.search(pattern, i)
            if size_search:
                size_values = i.split(":")[1]
                split_sz_vals = size_values.split(",")
                user_vals.append(split_sz_vals)

    flat_list = [item for sublist in user_vals for item in sublist]

    flat_list_float = convert_to_float(flat_list)

    out_rm_outliers = rm_sizes_outliers(flat_list_float, max_size, min_size)

    return Counter(out_rm_outliers)


def extract_item_size(aspects: List[Tuple], pattern: str) -> Optional[float]:
    for k in aspects:
        size_search = re.search(pattern, k[0])
        if size_search:
            size_value = k[1]
            try:
                size_value = float(size_value)
                return size_value
            except ValueError:
                # print(f"{size_value} is not a number")
                pass


def size_trx_per_user(df: pd.DataFrame, max_size: float, min_size: float, pattern=r'us shoe size$', vrtn_pattern=r'size',
                      aspects_col='ASPECTS',
                      vrtn_aspects_col='VRTN_ASPECTS') -> Counter:
    vrtn_aspects_list, aspects_list = [], []
    for idx, row in df.iterrows():
        if row[vrtn_aspects_col]:
            vrtn_aspects_list.append(extract_item_size(row[vrtn_aspects_col], vrtn_pattern))
        elif row[aspects_col]:
            aspects_list.append(extract_item_size(row[aspects_col], pattern))

    aspects_list.extend(vrtn_aspects_list)

    aspects_list = [i for i in aspects_list if i is not None]

    out_rm_outliers = rm_sizes_outliers(aspects_list, max_size, min_size)

    return Counter(out_rm_outliers)


def size_per_site_and_categ(df: pd.DataFrame, site_id: int, leaf_id: int, max_size: float, min_size: float,
                            history_type: str = 'filter', site_id_col: str = 'site_id', leaf_id_col='leaf_category_id',
                            user_id_col='user_id', secondary_regex=None) -> Optional[pd.DataFrame]:
    df = df[(df[site_id_col] == site_id) & (df[leaf_id_col] == leaf_id)]

    if history_type == 'filter':
        per_user_groupby = df.groupby([user_id_col]).apply(
            lambda x: size_filters_per_user(x['clean_aspects'], max_size, min_size))
    elif history_type == 'transactions':
        per_user_groupby = size_trx_per_user_parallel(df, user_id_col, max_size, min_size)
        # per_user_groupby = df.groupby([user_id_col]).apply(lambda x: size_trx_per_user(x, max_size, min_size))
    elif history_type == 'queries':
        per_user_groupby = df.groupby([user_id_col]).apply(
            lambda x: size_query_per_user(x['clean_query'], max_size, min_size, secondary_regex))
    else:
        print(f"history type can be either filter, transactions or queries, you provided {history_type}")
        return None

    df_per_user = pd.DataFrame({"user_id": per_user_groupby.index.astype(int), "sizes": per_user_groupby.values})

    # Remove users with empty dict of sizes
    df_per_user = df_per_user[df_per_user.sizes.astype(bool)]

    df_agg = agg_sizes(df_per_user, 'sizes', col_prefix=history_type)

    print(f"Output df shape for leaf_id: {leaf_id}, history type: {history_type} - {df_agg.shape}")

    return df_agg


def counter_multiply(counter, multiply_int: int):
    out_counter = counter.copy()
    for k in out_counter.keys():
        out_counter[k] = out_counter[k] * multiply_int
    return out_counter


def scale_size(sizes, scale) -> List:
    return [counter_multiply(i, scale) for i in sizes]


def merge_counter_sizes(counter_sizes: pd.Series):
    out = counter_sizes.iloc[0].copy()
    if len(counter_sizes) > 0:
        for c in counter_sizes.iloc[1:]:
            out += c
    return out


def merge_signals(df1: pd.DataFrame, df2: pd.DataFrame, df3=None):
    if df3 is not None:
        df = pd.concat([df1[['user_id', 'sizes']], df2[['user_id', 'sizes']], df3[['user_id', 'sizes']]])
    else:
        df = pd.concat([df1[['user_id', 'sizes']], df2[['user_id', 'sizes']]])

    df_merged = df.groupby('user_id').apply(lambda x: merge_counter_sizes(x['sizes'])).reset_index()
    df_merged.columns = ['user_id', 'sizes']

    print(f"DataFrame shape following merging sizes counts: {df_merged.shape}")

    df_agg = agg_sizes(df_merged, 'sizes', col_prefix='merged')

    return df_agg


def merge_and_add_leaf(df1: pd.DataFrame, leaf1: int, df2: pd.DataFrame, leaf2: int) -> pd.DataFrame:

    df1['leaf_id'] = leaf1
    df2['leaf_id'] = leaf2

    merged_df = pd.concat([df1, df2])

    # Moving leaf_id to the be the first column
    merged_df = merged_df[[merged_df.columns[-1]] + list(merged_df.columns[:-1])]

    return merged_df


def release_to_harvest(spark_df, spark: SparkSession, spark_queue: str):
    print(f'Converting Pandas DF to Spark DF and save as parquet for loading to harvest')
    spark_df.coalesce(10).write.mode("overwrite").parquet("/user/b_selling_research/gfuchs/temp/merged")
    print("Saving output to temporary HDFS location: /user/b_selling_research/gfuchs/temp/merged")

    print(f'Starting to load to harvest')
    import harvest as hv

    hv.executor.config('spark.yarn.queue', spark_queue)

    def load_df():
        import pandas as pd
        df = spark.read.parquet("/user/b_selling_research/gfuchs/temp/merged")
        return df

    udo = hv.udo.createFrom(load_df)
    res = udo.executeAndGet()
    print(f'harvest UDO result shape: {res.shape}')
    udo.save('sneakeres_testing',
             '1.0',
             override=True,
             spec={'featureSet': True,
                   'primaryKey': 'user_id'}
             )
    udo.release()


def size_trx_per_user_parallel(df: pd.DataFrame, user_id_col: str, max_size: float, min_size: float,
                               aspects_col='ASPECTS', vrtn_col='VRTN_ASPECTS', mp_cpus=35):

    pool = mp.Pool(processes=mp_cpus)

    df['aspects_vrtn_list'] = [i if type(i) == list else ['none'] for i in df[vrtn_col]]
    df['aspects_list'] = [i if type(i) == list else ['none'] for i in df[aspects_col]]

    df_aspects = df[df[vrtn_col].isna()]
    df_vrtn = df[~df[vrtn_col].isna()]
    print(df_aspects)
    print(df_vrtn)
    
    df_aspects['user_sizes'] = pool.map(extract_item_size_parallel_aspects, df_aspects.aspects_list)
    df_vrtn['user_sizes'] = pool.map(extract_item_size_parallel_vrtn, df_vrtn.aspects_vrtn_list)
    pool.close()
    df_union = pd.concat([df_vrtn, df_aspects])
    print(df_union.head(3))
    if None not in df_union['user_sizes']:
        print(df_union['user_sizes'])
        df_union['user_sizes'] = [i if max_size >= i >= min_size else None for i in df_union.user_sizes]

    df_union = df_union[~df_union.user_sizes.isna()]

    per_user_groupby = df_union.groupby(user_id_col)['user_sizes'].agg(Counter)

    return per_user_groupby


def extract_item_size_parallel_vrtn(aspects: List[Tuple]):
    for k in aspects:
        size_search = re.search(r'size', k[0])
        if size_search:
            size_value = k[1]
            try:
                size_value = float(size_value)
                return size_value
            except ValueError:
                pass


def extract_item_size_parallel_aspects(aspects: List[Tuple]):
    for k in aspects:
        size_search = re.search(r'us shoe size$', k[0])
        if size_search:
            size_value = k[1]
            try:
                size_value = float(size_value)
                return size_value
            except ValueError:
                pass


def save_to_hdfs_with_dt(df: pd.DataFrame, spark_session: SparkSession, hadoop_user: str) -> DataFrame:

    # Converting Counters to dict, as spark doesn't support Counters.
    df['sizes'] = [dict(i) for i in df['sizes']]

    spark_df = spark_session.createDataFrame(df)
    # Converting sizes column from Map to json string
    spark_df_json = spark_df.withColumn("sizes", F.to_json(F.col("sizes")))
    cur_date = datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d')

    hdfs_output_path = f"/user/{hadoop_user}/sneakers_size/snapshot/dt={cur_date}"
    spark_df_json.coalesce(10).write.mode("overwrite").parquet(hdfs_output_path)

    print(f"Output saved to HDFS: {hdfs_output_path}, time: {print_time()}")

    return spark_df_json




# Spark session

In [3]:
from pyspark.sql import SparkSession
import os
import socket

class SparkWrapper:

    def __init__(self, queue='hddq-exprce-perso-high-mem'):

        os.environ["PYSPARK_PYTHON"] = "/usr/share/anaconda3/python3.7/bin/python"
        self.queue = queue

    def init_session(self):

        spark = (
            SparkSession.builder.appName(os.environ['KRYLOV_PRINCIPAL'])
            .master("yarn")
            .config("spark.driver.maxResultSize", "10g")
            .config("spark.driver.host", socket.gethostbyname(socket.gethostname()))
            .config("spark.driver.port", "30202")
            .config("spark.executor.memoryOverhead", "8g") # In case a specific exectuer need extra memory, give this as extra.
            .config("spark.executor.cores", "8") # each worker has multiple exectuers, and each executer has multiple cores (=partitions), Dima suggested '8'
            .config("spark.driver.memory", "16g") # Dima set it to 24g, Nir = 8g
            .config("spark.executor.memory", "24g") # The regular RAM per exectuer (in contrast to memoryOverhead which is the extra).
            .config("spark.rdd.compress", True)
            .config("spark.network.timeout", "600s")
            .config("spark.executor.heartbeatInterval", "300s")
            .config("spark.sql.broadcastTimeout", "1200s")
            .config("spark.dynamicAllocation.enabled", "true") # take and give executors whenever needed - based on the actual data
            .config("spark.dynamicAllocation.minExecutors", 20) # 0 is problematic, yarn will use a worker with nothing
            .config("spark.dynamicAllocation.maxExecutors", 10000) # Nir suggested 200, as he said it is partitions/executer.cores (which equal to 3) - 512/3 < 200
            .config("spark.sql.shuffle.partitions", 500) # number of partitions after groupby/windowing, can be even 1000-2000 (per use-case)
            .config("spark.kryoserializer.buffer.max", "1g")
            .config("spark.yarn.queue", self.queue)
            .config("spark.speculation", False) # spark calcualte time per partition, when it is too long comparing to avg/median yarn will kill the partition, but the data is often skewed
            .config("spark.rpc.message.maxSize", 1024)
            .config("spark.pyspark.python", "/usr/share/anaconda3/python3.7/bin/python")
            .enableHiveSupport()
            .getOrCreate()
        )

        spark.sparkContext.setLogLevel("ERROR")
        spark.sparkContext.setCheckpointDir("checkpoint/")
        print("The executors' logs link:")
        print(f'https://apollo-rno-rm-2.vip.hadoop.ebay.com:50030/proxy/{spark.sparkContext.applicationId}')

        return spark


In [4]:
spark_wrapper = SparkWrapper(queue='hdlq-struct-default')
spark = spark_wrapper.init_session()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/04 12:14:10 WARN Utils: Service 'sparkDriver' could not bind on port 30202. Attempting port 30203.
23/07/04 12:14:10 WARN Utils: Service 'sparkDriver' could not bind on port 30203. Attempting port 30204.
23/07/04 12:14:10 WARN Utils: Service 'sparkDriver' could not bind on port 30204. Attempting port 30205.
23/07/04 12:14:11 WARN Utils: Service 'SparkUI' could not bind on port 30401. Attempting port 30402.
23/07/04 12:14:11 WARN Utils: Service 'SparkUI' could not bind on port 30402. Attempting port 30403.
23/07/04 12:14:11 WARN Utils: Service 'SparkUI' could not bind on port 30403. Attempting port 30404.
23/07/04 12:14:13 WARN HiveConf: DEPRECATED: hive.metastore.ds.retry.* no longer has any effect.  Use hive.hmshandler.retry.* instead
23/07/04 12:14:13 WARN HiveConf: HiveConf of name hive.metastore.local does not exist
23/07/04 12:14:13 WARN HiveConf:

The executors' logs link:
https://apollo-rno-rm-2.vip.hadoop.ebay.com:50030/proxy/application_1687661240187_983082


In [ ]:
srp_event_fact = spark.table("ACCESS_VIEWS.SRCH_SRP_TOOL_EVENT_FACT").coalesce(10000)

In [6]:
srp_time_delta = 90
srp_dates = delta_to_dates(srp_time_delta)

Given time_delta: 90, updated start_dt: 2023-01-13 and end_dt: 2023-04-13


In [9]:
df_srp = basic_srp_processing(table=srp_event_fact,
                              start_dt=srp_dates['start_dt'],
                              end_dt=srp_dates['end_dt'],
                              site_id=0,
                              leaf_ids=[15709, 95672])

In [12]:
df_queries_rm_dups = filter_srp_by_col(df_srp, "clean_query")

In [21]:
df_filters_rm_dups = filter_srp_by_col(df_srp, "clean_aspects")

In [15]:
df_queries_regex = filter_by_regex(df_queries_rm_dups,
                                           primary_regex=r'(size|sz) (\d+(?:\.\d+)?)',
                                           secondary_regex=r"\d+\.5")

In [13]:
hadoop_user = "b_selling_research"
tmp_output = f"/user/{hadoop_user}/sneakers_size/temp"

In [35]:
# df_queries_regex.repartition(20, "user_id").write.mode("overwrite").parquet(f"{tmp_output}/queries")

In [36]:
# df_filters_rm_dups.repartition(20, "user_id").write.mode("overwrite").parquet(f"{tmp_output}/filters")

# Load pre-calucatd data

The job was to created the data was executed automatcilly on 15-Apr, the loading to the notebook was done at 16-Apr

In [378]:
tmp_output

'/user/b_selling_research/sneakers_size/temp'

In [ ]:
/data/ebay/data/gfuchs/temp/for_ortal_size/filters_df
/data/ebay/data/gfuchs/temp/for_ortal_size/queries_df
/data/ebay/data/gfuchs/temp/for_ortal_size/trx_rtrn_df

In [9]:
!ls /data/ebay/data/gfuchs/temp/for_ortal_size/filters_df

/data/ebay/data/gfuchs/temp/for_ortal_size/filters_df


In [5]:
filters_df = pd.read_parquet(f"/data/ebay/data/gfuchs/temp/for_ortal_size/filters_df")

In [6]:
queries_df=pd.read_parquet(f"/data/ebay/data/gfuchs/temp/for_ortal_size/queries_df")

In [7]:
trx_df=pd.read_parquet(f"/data/ebay/data/gfuchs/temp/for_ortal_size/trx_rtrn_df")

In [8]:
queries_df.shape, filters_df.shape, trx_df.shape

((11303795, 7), (18043549, 7), (7267125, 13))

In [9]:
filters_df.session_start_dt.max(), filters_df.session_start_dt.min()

(datetime.date(2023, 4, 13), datetime.date(2023, 1, 15))

In [10]:
queries_df.session_start_dt.max(), queries_df.session_start_dt.min()

(datetime.date(2023, 4, 13), datetime.date(2023, 1, 15))

In [11]:
trx_df.CREATED_DT.max(), trx_df.CREATED_DT.min()

(datetime.date(2023, 4, 16), datetime.date(2022, 4, 17))

### Add size for each source per: user, date, category:

#### Basic_srp_processing - input data from directories 

In [12]:
filters_df[filters_df['user_id']==5242].head(3)

,site_id,session_skey,user_id,session_start_dt,clean_aspects,clean_query,dominant_category_id
5947038,0,38850349936040,5242,2023-02-10,us shoe size:10,air jordan golf,15709
6142121,0,38890014991530,5242,2023-03-28,us shoe size:10,air zoom victory,15709


In [13]:
filters_df[filters_df['user_id']==77].head(3)

,site_id,session_skey,user_id,session_start_dt,clean_aspects,clean_query,dominant_category_id


In [14]:
queries_df[queries_df['user_id']==77].head(3)

,site_id,session_skey,user_id,session_start_dt,clean_aspects,clean_query,dominant_category_id


In [107]:
trx_df[trx_df['BUYER_ID']==77].head(3)

,ITEM_ID,TRANSACTION_ID,ITEM_RETURNED,ITEM_VRTN_ID,BUYER_ID,AUCT_END_DT,LEAF_CATEG_ID,SITE_ID,SELLER_ID,ITEM_PRICE,CREATED_DT,ASPECTS,VRTN_ASPECTS,ASPECTS_NEWF,VRTN_ASPECTS_NEWF,aspects_vrtn_list,aspects_list
930768,313925739614,1216777870021,0,-999,77,2022-04-28,95672,0,102730768,93.15,2022-04-28,"[[color, gray], [activity, running], [uom1, si...",None,"[(color, gray), (activity, running), (uom1, si...",None,[none],"[(color, gray), (activity, running), (uom1, si..."
1298693,334554953329,2008940978014,0,-999,77,2022-10-23,95672,0,574333378,17.59,2022-10-23,"[[country/region of manufacture, vietnam], [co...",None,"[(country/region of manufacture, vietnam), (co...",None,[none],"[(country/region of manufacture, vietnam), (co..."
1340885,334560819670,2007463741014,0,-999,77,2022-10-16,95672,0,574333378,43.99,2022-10-16,"[[country/region of manufacture, vietnam], [co...",None,"[(country/region of manufacture, vietnam), (co...",None,[none],"[(country/region of manufacture, vietnam), (co..."


In [17]:
trx_df.loc[1,'ASPECTS' ]

array([array(['color', 'black'], dtype=object),
       array(['uom1', 'size 10'], dtype=object),
       array(['type', 'athletic'], dtype=object),
       array(['miscatscore', '12'], dtype=object),
       array(['maturityscore', '8'], dtype=object),
       array(['character', 'snike'], dtype=object),
       array(['release year', '2019'], dtype=object),
       array(['producttitle',
       'size 10 - nike air zoom pegasus 36 tcu purple 2019'], dtype=object),
       array(['mergednamespacenames', 'us shoe size'], dtype=object),
       array(['theme', 'colorful'], dtype=object),
       array(['model', 'nike pegasus'], dtype=object),
       array(['catrecoid_3', '95672'], dtype=object),
       array(['department', 'men'], dtype=object),
       array(['catrecoid_2', '24541'], dtype=object),
       array(['upper material', 'mesh'], dtype=object),
       array(['catrecoid_1', '15709'], dtype=object),
       array(['brand', 'nike'], dtype=object),
       array(['performance/activity', 'runnin

In [16]:
user_id_col = "user_id"
max_size = 16
min_size = 4.5
date_col = 'session_start_dt'
categ_col = 'dominant_category_id'

### Filters 

#### Aggregate in date level ( level up from session level) and extract size from filter : 

In [17]:
filters_per_user_date_categ_groupby = filters_df.groupby([categ_col, user_id_col, date_col]).apply(
    lambda x: size_filters_per_user(x['clean_aspects'], max_size, min_size))

In [18]:
filters_per_user_date_categ_groupby.head(5)

dominant_category_id  user_id  session_start_dt
15709                 192      2023-02-02           {8.5: 1}
                      376      2023-02-20           {9.5: 1}
                      526      2023-03-26          {12.0: 2}
                      847      2023-01-23          {10.0: 1}
                      862      2023-01-22           {9.0: 1}
dtype: object

In [19]:
filters_per_user_date_categ_groupby=filters_per_user_date_categ_groupby.to_frame().reset_index()

In [20]:
filters_per_user_date_categ_groupby[filters_per_user_date_categ_groupby['user_id']==77]

,dominant_category_id,user_id,session_start_dt,0


In [21]:
filters_per_user_date_categ_groupby.columns

Index(['dominant_category_id', 'user_id', 'session_start_dt', 0], dtype='object')

In [22]:
filters_per_user_date_categ_groupby.rename(columns = { 0:'user_sizes'}, inplace = True)
#rename to szes and not user_sizes

### Queries: 

In [ ]:
queries_df.head(3).groupby([categ_col, user_id_col, date_col]).apply(
    lambda x: size_query_per_user(x['clean_query'], max_size, min_size))

In [ ]:
def extract_query_size(query: str, primary_regex=r'(size|sz) (\d+(?:\.\d+)?)', secondary_regex=None):
    find_size = re.findall(primary_regex, query)

In [42]:
primary_regex=r'(size|sz) (\d+(?:\.\d+)?)'
query='1985 jordan size 13'
re.findall(primary_regex, query)

[('size', '13')]

In [40]:
queries_df.head(3)

,site_id,session_skey,user_id,session_start_dt,clean_aspects,clean_query,dominant_category_id
0,0,38840641053920,2072725289,2023-01-30,,1985 jordan size 13,15709
1,0,38862920543100,2072725289,2023-02-25,,1985 jordan size 13,15709
2,0,38845168410750,1459699660,2023-02-04,,1985 size 9,15709


In [23]:
queries_per_user_date_categ_groupby = queries_df.groupby([categ_col, user_id_col, date_col]).apply(
    lambda x: size_query_per_user(x['clean_query'], max_size, min_size))

In [24]:
queries_per_user_date_categ_groupby=queries_per_user_date_categ_groupby.to_frame().reset_index()

In [25]:
queries_per_user_date_categ_groupby.rename(columns = { 0:'user_sizes'}, inplace = True)
#rename to szes and not user_sizes

In [26]:
queries_per_user_date_categ_groupby.columns

Index(['dominant_category_id', 'user_id', 'session_start_dt', 'user_sizes'], dtype='object')

In [27]:
queries_per_user_date_categ_groupby.head(1)

,dominant_category_id,user_id,session_start_dt,user_sizes
0,15709,376,2023-02-20,{}


In [28]:
queries_per_user_date_categ_groupby[queries_per_user_date_categ_groupby['user_id']==77] 

,dominant_category_id,user_id,session_start_dt,user_sizes


In [22]:
filters_df.loc[0:10,'clean_aspects']

0     brand:nike;performance/activity:cross training...
1                                        us shoe size:4
2                                       us shoe size:14
3                                       us shoe size:11
4                       color:multicolor;us shoe size:9
5                                     us shoe size:10.5
6                                    us shoe size:8.5,9
7                                        us shoe size:8
8                               us shoe size:13.5,6,6.5
9                                    us shoe size:8.5,9
10                                      us shoe size:12
Name: clean_aspects, dtype: object

### Transactions : 

In [196]:
trx_df[trx_df.ASPECTS.isna()]

,ITEM_ID,TRANSACTION_ID,ITEM_RETURNED,ITEM_VRTN_ID,BUYER_ID,AUCT_END_DT,LEAF_CATEG_ID,SITE_ID,SELLER_ID,ITEM_PRICE,CREATED_DT,ASPECTS,VRTN_ASPECTS,aspects_vrtn_list
520,195504872548,1978807966009,0,-999,1435351457,2022-12-05,15709,0,391323004,280.00,2022-12-05,None,None,[none]
551,204041005938,2176900380010,0,-999,2206483541,2022-12-05,95672,0,1980813813,30.00,2022-12-05,None,None,[none]
568,204113682815,2164333630010,0,504657354394,1286448044,2022-12-06,95672,0,52223114,39.99,2022-10-15,None,"[[shoe width, b], [us shoe size, 5]]",[none]
786,266006887543,2951003017016,0,-999,2442225276,2022-12-05,95672,0,1070913939,259.99,2022-12-05,None,None,[none]
850,275536183671,2588827506017,0,-999,1106693586,2022-12-05,95672,0,1754541522,15.00,2022-12-05,None,None,[none]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7265414,354035370638,1340254431022,0,-999,2212465212,2022-05-02,95672,0,2364236833,27.00,2022-05-02,None,None,[none]
7265439,354383816121,1391474081022,0,-999,2427779087,2022-12-05,15709,0,2414398510,525.00,2022-12-05,None,None,[none]
7266285,225245232860,2742107095012,0,-999,792077174,2022-12-05,15709,0,1659716859,77.00,2022-12-05,None,None,[none]
7266437,266003513290,2950957472016,0,-999,2422735625,2022-12-05,15709,0,15617587,25.00,2022-12-05,None,None,[none]


In [29]:
def change_aspects_format(values: list ):
    #for each aspect val: change to list of tuples in tuple 
    aspects=[]
    for i in values: 
        aspects.append(tuple(i))
        
    return aspects

#### Format change of trx aspects- due to parquet format:

In [30]:
trx_df['ASPECTS_NEWF']=trx_df['ASPECTS'].apply(lambda x : change_aspects_format(x) if x != None else x )

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


In [31]:
trx_df['VRTN_ASPECTS_NEWF']=trx_df['VRTN_ASPECTS'].apply(lambda x : change_aspects_format(x) if x != None else x )

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


In [32]:
trx_df['ASPECTS_NEWF'][0]

[('country/region of manufacture', 'china'),
 ('customized', 'no'),
 ('color', 'multicolor'),
 ('greenscore', '100'),
 ('brand type', 'contemporary'),
 ('uom1', 'size 9.5'),
 ('type', 'athletic'),
 ('shoe width', 'standard'),
 ("us shoe size (men's)", '9.5'),
 ('miscatscore', '5'),
 ('release year', '2010'),
 ('mergednamespacenames', 'release year'),
 ('model', 'nike sc'),
 ('department', 'men'),
 ('upper material', 'nylon'),
 ('catrecoid_1', '15709'),
 ('brand', 'nike'),
 ('performance/activity', 'gym & training'),
 ('miscatscore_cf_v1', '5'),
 ('p2sprobability', '234083'),
 ('ks', 'shoesize/us_men#9.5'),
 ('us shoe size', '9.5'),
 ('catrecoscore_1', '99'),
 ('vintage', 'no'),
 ('miscatscore_v1', '5'),
 ('product line', 'nike trainer'),
 ('year manufactured', '2010'),
 ('style', 'sneaker'),
 ('shoe shaft style', 'low top')]

In [33]:
trx_df.head(2)

,ITEM_ID,TRANSACTION_ID,ITEM_RETURNED,ITEM_VRTN_ID,BUYER_ID,AUCT_END_DT,LEAF_CATEG_ID,SITE_ID,SELLER_ID,ITEM_PRICE,CREATED_DT,ASPECTS,VRTN_ASPECTS,ASPECTS_NEWF,VRTN_ASPECTS_NEWF
0,114260471912,2310012098001,0,-999,56595811,2022-08-06,15709,0,120908130,275.00,2022-08-06,"[[country/region of manufacture, china], [cust...",None,"[(country/region of manufacture, china), (cust...",None
1,114530153783,2321895259001,0,-999,2453767164,2022-10-02,15709,0,1619572747,89.99,2022-10-02,"[[color, black], [uom1, size 10], [type, athle...",None,"[(color, black), (uom1, size 10), (type, athle...",None


In [ ]:
#extract size from trx: 

#####  trx processing to return size per categ and date of trx, reduce mp_cpus=4 to 4:

In [ ]:
##### modified below def 

In [34]:
from datetime import date
from collections import Counter 

def size_trx_per_user_parallel(df: pd.DataFrame, leaf_id:int, user_id_col: str, trx_dt: date,  max_size: float, min_size: float,
                               aspects_col='ASPECTS', vrtn_col='VRTN_ASPECTS', mp_cpus=6):

    pool = mp.Pool(processes=mp_cpus)

    df['aspects_vrtn_list'] = [i if type(i) == list else ['none'] for i in df[vrtn_col]]
    df['aspects_list'] = [i if type(i) == list else ['none'] for i in df[aspects_col]]

    df_aspects = df[df[vrtn_col].isna()]
    df_vrtn = df[~df[vrtn_col].isna()]

    df_aspects['user_sizes'] = pool.map(extract_item_size_parallel_aspects, df_aspects.aspects_list)
    df_vrtn['user_sizes'] = pool.map(extract_item_size_parallel_vrtn, df_vrtn.aspects_vrtn_list)     
    pool.close()
    
    df_union = pd.concat([df_vrtn, df_aspects])   
    df_union['user_sizes'] = np.where((df_union.user_sizes >= min_size) & (df_union.user_sizes <= max_size), df_union.user_sizes, None)
    #df_union['user_sizes'] = [i if max_size >= i >= min_size else None for i in df_union.user_sizes]
    #df_union = df_union[~df_union.user_sizes.isna()]
    df_union = df_union[df_union.user_sizes.notnull()]

    
    #per_user_groupby = df_union.groupby([user_id_col,leaf_id,trx_dt])['user_sizes'].agg(Counter)
    per_user_groupby = df_union[df_union['user_sizes'].notnull()].groupby([user_id_col, leaf_id, trx_dt])['user_sizes'].agg(Counter).to_frame().reset_index()
    #remove empty sizes: 
    per_user_groupby= per_user_groupby[per_user_groupby.user_sizes.astype(bool)]
    
    #df_per_user = pd.DataFrame({"user_id": per_user_groupby.user_id_col.astype(int), "sizes": per_user_groupby.user_sizes})
    return per_user_groupby

##### test function error on example:

In [69]:
df=trx_df.head(500).copy()
aspects_col='ASPECTS_NEWF'
vrtn_col='VRTN_ASPECTS_NEWF'
df['aspects_vrtn_list'] = [i if type(i) == list else ['none'] for i in df[vrtn_col]]
df['aspects_list'] = [i if type(i) == list else ['none'] for i in df[aspects_col]]

In [70]:
df_aspects = df[df[vrtn_col].isna()]
df_vrtn = df[~df[vrtn_col].isna()]

In [71]:
mp_cpus=3
pool = mp.Pool(processes=mp_cpus)
df_aspects['user_sizes'] = pool.map(extract_item_size_parallel_aspects, df_aspects.aspects_list)
df_vrtn['user_sizes'] = pool.map(extract_item_size_parallel_vrtn, df_vrtn.aspects_vrtn_list)
pool.close()
df_union = pd.concat([df_vrtn, df_aspects])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [72]:
df_union['user_sizes'] = [i if max_size >= i >= min_size else None for i in df_union.user_sizes]
df_union = df_union[~df_union.user_sizes.isna()]

In [84]:
per_user_groupby = df_union.groupby([user_id_col,leaf_id])['user_sizes'].agg(Counter).to_frame().reset_index()

In [85]:
per_user_groupby.columns

Index(['BUYER_ID', 'LEAF_CATEG_ID', 'user_sizes'], dtype='object')

In [86]:
per_user_groupby

,BUYER_ID,LEAF_CATEG_ID,user_sizes
0,82887,15709,{10.5: 1}
1,1897241,15709,{12.0: 1}
2,2398687,15709,{8.0: 1}
3,4345742,15709,{13.0: 1}
4,4898512,15709,{11.5: 1}
...,...,...,...
468,2497810660,95672,{7.5: 1}
469,2505156331,15709,{11.5: 1}
470,2505279760,95672,{6.5: 1}
471,2510981216,15709,{9.0: 1}


In [88]:
per_user_groupby= per_user_groupby[per_user_groupby.user_sizes.astype(bool)]

In [89]:
per_user_groupby

,BUYER_ID,LEAF_CATEG_ID,user_sizes
0,82887,15709,{10.5: 1}
1,1897241,15709,{12.0: 1}
2,2398687,15709,{8.0: 1}
3,4345742,15709,{13.0: 1}
4,4898512,15709,{11.5: 1}
...,...,...,...
468,2497810660,95672,{7.5: 1}
469,2505156331,15709,{11.5: 1}
470,2505279760,95672,{6.5: 1}
471,2510981216,15709,{9.0: 1}


In [75]:
per_user_groupby

BUYER_ID    LEAF_CATEG_ID
82887       15709            {10.5: 1}
1897241     15709            {12.0: 1}
2398687     15709             {8.0: 1}
4345742     15709            {13.0: 1}
4898512     15709            {11.5: 1}
                               ...    
2497810660  95672             {7.5: 1}
2505156331  15709            {11.5: 1}
2505279760  95672             {6.5: 1}
2510981216  15709             {9.0: 1}
2514371622  15709            {12.0: 1}
Name: user_sizes, Length: 473, dtype: object

#### Transactions: extract size from user trx and aggregate per buyer_id, leaf, created dt : 

In [35]:
user_id_col = "user_id"
max_size = 16
min_size = 4.5
date_col = 'session_start_dt'
categ_col = 'dominant_category_id'

In [36]:
user_id_col='BUYER_ID'
aspects_col='ASPECTS_NEWF'
vrtn_col='VRTN_ASPECTS_NEWF'
leaf_id= 'LEAF_CATEG_ID' 
trx_dt='CREATED_DT'
trx_per_user_date_categ_groupby =size_trx_per_user_parallel(trx_df ,leaf_id, user_id_col,trx_dt, max_size, min_size,  aspects_col, vrtn_col , mp_cpus=8)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [37]:
trx_per_user_date_categ_groupby.head(3)

,BUYER_ID,LEAF_CATEG_ID,CREATED_DT,user_sizes
0,39,15709,2022-06-06,{9.5: 1}
1,60,15709,2022-05-15,{12.0: 1}
2,60,15709,2022-09-21,{12.5: 1}


In [38]:
trx_per_user_date_categ_groupby[trx_per_user_date_categ_groupby['BUYER_ID']==5242].sort_values(by='BUYER_ID', ascending=False)

,BUYER_ID,LEAF_CATEG_ID,CREATED_DT,user_sizes
91,5242,15709,2023-02-16,{10.0: 1}


#### Remove empty values of size:  

In [39]:
queries_per_user_date_categ_groupby= queries_per_user_date_categ_groupby[queries_per_user_date_categ_groupby.user_sizes.astype(bool)]
filters_per_user_date_categ_groupby= filters_per_user_date_categ_groupby[filters_per_user_date_categ_groupby.user_sizes.astype(bool)]
trx_per_user_date_categ_groupby= trx_per_user_date_categ_groupby[trx_per_user_date_categ_groupby.user_sizes.astype(bool)] 

##### Save data as pickle: per source have: 'dominant_category_id', 'user_id', 'session_start_dt', 'user_sizes'

In [40]:
filters_per_user_date_categ_groupby.to_pickle('/data/ebay/data/olivyatan/size_data/filters_per_user_3m_30723.pkl')
queries_per_user_date_categ_groupby.to_pickle('/data/ebay/data/olivyatan/size_data/queries_per_user_3m_30723.pkl')
trx_per_user_date_categ_groupby.to_pickle('/data/ebay/data/olivyatan/size_data/trx_per_user_3m_30723.pkl')

#### Change size format to str in order to save as parquet: 

In [425]:
type(z_df.loc[0, 'user_sizes'])

str

In [ ]:
#Convert from counter to str: 
z = [str(dict(i)) for i in trx_per_user_date_categ_groupby.user_sizes.iloc[:5]]

In [476]:
#convert to str from counter:
[Counter(eval(i)) for i in trx_per_user_date_categ_groupby.user_sizes.iloc[:5]]

[Counter({9.5: 1}),
 Counter({12.0: 1}),
 Counter({12.5: 1}),
 Counter({7.0: 1}),
 Counter({13.0: 1})]

In [479]:
#convert to str to align str format: 

In [426]:
trx_per_user_date_categ_groupby.user_sizes=[str(dict(i)) for i in trx_per_user_date_categ_groupby.user_sizes]

In [436]:
filters_per_user_date_categ_groupby.user_sizes=[str(dict(i)) for i in filters_per_user_date_categ_groupby.user_sizes]

In [438]:
queries_per_user_date_categ_groupby.user_sizes=[str(dict(i)) for i in queries_per_user_date_categ_groupby.user_sizes]

In [439]:
type(queries_per_user_date_categ_groupby.loc[0, 'user_sizes'])

str

In [ ]:
#### convert to counter from str after reading data: 

In [8]:
filters_per_user_date_categ_groupby=pd.read_parquet('/data/ebay/data/olivyatan/size_data/filters_per_user_3m_19.4.23.parquet')

In [9]:
queries_per_user_date_categ_groupby=pd.read_parquet('/data/ebay/data/olivyatan/size_data/queries_per_user_3m_19.4.23.parquet')

In [10]:
trx_per_user_date_categ_groupby=pd.read_parquet('/data/ebay/data/olivyatan/size_data/trx_per_user_3m_19.4.23.parquet')

In [11]:
trx_per_user_date_categ_groupby.user_sizes=[Counter(eval(i)) for i in trx_per_user_date_categ_groupby.user_sizes]

In [12]:
filters_per_user_date_categ_groupby.user_sizes=[Counter(eval(i)) for i in filters_per_user_date_categ_groupby.user_sizes]

In [13]:
queries_per_user_date_categ_groupby.user_sizes=[Counter(eval(i)) for i in queries_per_user_date_categ_groupby.user_sizes]

In [14]:
type(queries_per_user_date_categ_groupby.loc[0, 'user_sizes'])

collections.Counter

In [ ]:
### Save size data location: 

In [378]:
pwd

'/data/ebay/data/olivyatan'

In [ ]:
###Remove empty 

In [442]:
!ls /data/ebay/data/olivyatan/

df_data  df_data.pkl  df_data_ex_na.pkl  iob_data  msku_pg  size_data  temp


In [ ]:
##save data to parquet: per user date categ have counter of sizes. 

In [440]:
filters_per_user_date_categ_groupby.to_parquet('/data/ebay/data/olivyatan/size_data/filters_per_user_3m_19.4.23.parquet')

In [441]:
queries_per_user_date_categ_groupby.to_parquet('/data/ebay/data/olivyatan/size_data/queries_per_user_3m_19.4.23.parquet')

In [428]:
trx_per_user_date_categ_groupby.to_parquet('/data/ebay/data/olivyatan/size_data/trx_per_user_3m_19.4.23.parquet')

In [373]:
ls /data/ebay/data/olivyatan/

df_data  df_data.pkl  df_data_ex_na.pkl  iob_data/  msku_pg/  size_data/  temp/


In [372]:
mkdir /data/ebay/data/olivyatan/size_data

In [15]:
queries_per_user_date_categ_groupby= queries_per_user_date_categ_groupby[queries_per_user_date_categ_groupby.user_sizes.astype(bool)]
filters_per_user_date_categ_groupby= filters_per_user_date_categ_groupby[filters_per_user_date_categ_groupby.user_sizes.astype(bool)]
trx_per_user_date_categ_groupby= trx_per_user_date_categ_groupby[trx_per_user_date_categ_groupby.user_sizes.astype(bool)] 

### Label: 

####  Add agg stats for transactions to prepare label: 

In [41]:
#need it for label creation of trx:
df_agg_trx_per_user = agg_sizes(trx_per_user_date_categ_groupby, 'user_sizes', col_prefix='transactions')

In [42]:
df_agg_trx_per_user.head(1)

,BUYER_ID,LEAF_CATEG_ID,CREATED_DT,user_sizes,transactions_std,transactions_mean,transactions_median,transactions_mode,transactions_maxMinDiff,transactions_countOfEvidence
0,39,15709,2022-06-06,{9.5: 1},0.0,9.5,9.5,9.5,0.0,1


In [23]:
type(df_agg_trx_per_user.loc[0, 'user_sizes'])

collections.Counter

In [ ]:
df_agg_trx_per_user.user_sizes=[Counter(eval(i)) for i in df_agg_trx_per_user.user_sizes]

In [43]:
df_agg_trx_per_user.head(3)

,BUYER_ID,LEAF_CATEG_ID,CREATED_DT,user_sizes,transactions_std,transactions_mean,transactions_median,transactions_mode,transactions_maxMinDiff,transactions_countOfEvidence
0,39,15709,2022-06-06,{9.5: 1},0.0,9.5,9.5,9.5,0.0,1
1,60,15709,2022-05-15,{12.0: 1},0.0,12.0,12.0,12.0,0.0,1
2,60,15709,2022-09-21,{12.5: 1},0.0,12.5,12.5,12.5,0.0,1


In [44]:
df_agg_trx_per_user.shape

(5753576, 10)

#### Most recent transaction per user and categ: 

In [45]:
df_agg_trx_per_user.rename(columns={ "BUYER_ID": "user_id","LEAF_CATEG_ID": "leaf_id"},inplace=True)

In [46]:
df_agg_trx_per_user['row_number'] = df_agg_trx_per_user.sort_values(['user_id','leaf_id','CREATED_DT'], ascending=[True,True,False]) \
             .groupby(['user_id','leaf_id']) \
             .cumcount() + 1

In [47]:
df_agg_trx_per_user.shape

(5753576, 11)

In [48]:
df_agg_trx_per_user.head(3)

,user_id,leaf_id,CREATED_DT,user_sizes,transactions_std,transactions_mean,transactions_median,transactions_mode,transactions_maxMinDiff,transactions_countOfEvidence,row_number
0,39,15709,2022-06-06,{9.5: 1},0.0,9.5,9.5,9.5,0.0,1,1
1,60,15709,2022-05-15,{12.0: 1},0.0,12.0,12.0,12.0,0.0,1,6
2,60,15709,2022-09-21,{12.5: 1},0.0,12.5,12.5,12.5,0.0,1,5


In [49]:
# we keep as label only rows where there is recent transaction with 1 size , for these rows we will add training data of 4 features of 
# size per user and categ: 
#remove transactions cases of most recent and mean is equal to median, otherwise it means in same day of most recent trx 2 different sizes 

In [50]:
label_trx= df_agg_trx_per_user[ (df_agg_trx_per_user['row_number']==1) &  (df_agg_trx_per_user['transactions_countOfEvidence']>=1) &  \
                               (df_agg_trx_per_user['transactions_mean']==df_agg_trx_per_user['transactions_mode']) ]

In [51]:
label_trx.head(2)

,user_id,leaf_id,CREATED_DT,user_sizes,transactions_std,transactions_mean,transactions_median,transactions_mode,transactions_maxMinDiff,transactions_countOfEvidence,row_number
0,39,15709,2022-06-06,{9.5: 1},0.0,9.5,9.5,9.5,0.0,1,1
6,60,15709,2023-04-05,{13.0: 1},0.0,13.0,13.0,13.0,0.0,1,1


In [52]:
 label_trx.shape

(3201768, 11)

In [53]:
 label_trx.shape

(3201768, 11)

In [54]:
label_trx[label_trx['transactions_countOfEvidence']>1].head(3)

,user_id,leaf_id,CREATED_DT,user_sizes,transactions_std,transactions_mean,transactions_median,transactions_mode,transactions_maxMinDiff,transactions_countOfEvidence,row_number
50,2689,15709,2022-05-09,{12.0: 2},0.0,12.0,12.0,12.0,0.0,2,1
64,3751,15709,2022-12-31,{10.0: 3},0.0,10.0,10.0,10.0,0.0,3,1
104,6246,95672,2022-05-03,{8.0: 2},0.0,8.0,8.0,8.0,0.0,2,1


In [55]:
label_trx.user_id.nunique()

2991975

In [56]:
df_agg_trx_per_user[df_agg_trx_per_user.user_id==60]

,user_id,leaf_id,CREATED_DT,user_sizes,transactions_std,transactions_mean,transactions_median,transactions_mode,transactions_maxMinDiff,transactions_countOfEvidence,row_number
1,60,15709,2022-05-15,{12.0: 1},0.0,12.0,12.0,12.0,0.0,1,6
2,60,15709,2022-09-21,{12.5: 1},0.0,12.5,12.5,12.5,0.0,1,5
3,60,15709,2022-11-20,{7.0: 1},0.0,7.0,7.0,7.0,0.0,1,4
4,60,15709,2023-01-04,{13.0: 1},0.0,13.0,13.0,13.0,0.0,1,3
5,60,15709,2023-02-18,{5.0: 1},0.0,5.0,5.0,5.0,0.0,1,2
6,60,15709,2023-04-05,{13.0: 1},0.0,13.0,13.0,13.0,0.0,1,1
7,60,95672,2022-05-14,{8.0: 1},0.0,8.0,8.0,8.0,0.0,1,8
8,60,95672,2022-12-05,{6.5: 1},0.0,6.5,6.5,6.5,0.0,1,7
9,60,95672,2022-12-22,{6.5: 1},0.0,6.5,6.5,6.5,0.0,1,6
10,60,95672,2023-01-08,{6.5: 1},0.0,6.5,6.5,6.5,0.0,1,5


In [57]:
queries_per_user_date_categ_groupby.rename(columns={"user_sizes": "sizes"}, inplace=True)
filters_per_user_date_categ_groupby.rename(columns={"user_sizes": "sizes"}, inplace=True)
trx_per_user_date_categ_groupby.rename(columns={"user_sizes": "sizes"}, inplace=True) 
label_trx.rename(columns={"user_sizes": "sizes"}, inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
df_agg_trx_per_user_rml.rename(columns={"user_sizes": "sizes"}, inplace=True)

In [58]:
label_trx[label_trx.user_id==60].head(2)

,user_id,leaf_id,CREATED_DT,sizes,transactions_std,transactions_mean,transactions_median,transactions_mode,transactions_maxMinDiff,transactions_countOfEvidence,row_number
6,60,15709,2023-04-05,{13.0: 1},0.0,13.0,13.0,13.0,0.0,1,1
14,60,95672,2023-03-23,{6.5: 1},0.0,6.5,6.5,6.5,0.0,1,1


In [71]:
queries_per_user_date_categ_groupby[queries_per_user_date_categ_groupby['user_id']==5242]

,dominant_category_id,user_id,session_start_dt,sizes


In [59]:
filters_per_user_date_categ_groupby[filters_per_user_date_categ_groupby['user_id']==5242]

,dominant_category_id,user_id,session_start_dt,sizes
26,15709,5242,2023-02-10,{10.0: 1}
27,15709,5242,2023-03-28,{10.0: 1}


In [60]:
trx_per_user_date_categ_groupby.head(1)

,BUYER_ID,LEAF_CATEG_ID,CREATED_DT,sizes
0,39,15709,2022-06-06,{9.5: 1}


#### Remove leakage : srcs dates before trx label date 

##### examination on transactions:

In [61]:
trx_per_user_date_categ_groupby.shape

(5753576, 4)

In [74]:
trx_per_user_date_categ_groupby_rml = pd.merge(trx_per_user_date_categ_groupby, 
                                    label_trx[["user_id","leaf_id","CREATED_DT","transactions_mode" ]], 
                                   left_on=["BUYER_ID","LEAF_CATEG_ID"], 
                                   right_on=[ "user_id","leaf_id"],
                                   how='left').rename(columns={'CREATED_DT_x': 'date', 'CREATED_DT_y': 'label_CREATED_DT'})



In [76]:
trx_per_user_date_categ_groupby_rml =trx_per_user_date_categ_groupby_rml[trx_per_user_date_categ_groupby_rml['date']
                                                                         < trx_per_user_date_categ_groupby_rml['label_CREATED_DT']]

In [159]:
trx_per_user_date_categ_groupby_rml.head(1)

,BUYER_ID,LEAF_CATEG_ID,date,sizes,user_id,leaf_id,label_CREATED_DT
1,60,15709,2022-05-15,{12.0: 1},60,15709,2023-04-05


In [77]:
trx_per_user_date_categ_groupby_rml.shape

(2328873, 8)

#### Remove leakage for all 3 sources:

In [61]:
def filter_src_dt_before_label(df_src: pd.DataFrame,df_label: pd.DataFrame ,  src_dt_col_name:str,label_dt_col_name:str,label_trx_size_col:str,
                               left_on_cols: list , right_on_cols: list ):
    
    df_src.rename(columns={src_dt_col_name: 'date'}, inplace=True) 
    df_label.rename(columns={label_dt_col_name: 'label_trx_dt'}, inplace=True) 
    df_label.rename(columns={label_trx_size_col: 'label_trx_size'}, inplace=True) 
    size_src_per_user_groupby_rml = pd.merge(df_src, 
                                    df_label, 
                                    left_on=left_on_cols, 
                                    right_on=right_on_cols,
                                    how='left')
    
    size_src_per_user_groupby_rml =size_src_per_user_groupby_rml[size_src_per_user_groupby_rml['date']< size_src_per_user_groupby_rml['label_trx_dt']]
    return size_src_per_user_groupby_rml

In [62]:
trx_per_user_date_categ_groupby_rml= filter_src_dt_before_label( trx_per_user_date_categ_groupby,label_trx[["user_id","leaf_id","CREATED_DT","transactions_mode" ]], 
                                                          'CREATED_DT', 'CREATED_DT',"transactions_mode",
                                                          left_on_cols=["BUYER_ID","LEAF_CATEG_ID"] , right_on_cols=[ "user_id","leaf_id"]  )

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [63]:
trx_per_user_date_categ_groupby_rml[trx_per_user_date_categ_groupby_rml.BUYER_ID==60]

,BUYER_ID,LEAF_CATEG_ID,date,sizes,user_id,leaf_id,label_trx_dt,label_trx_size
1,60,15709,2022-05-15,{12.0: 1},60,15709,2023-04-05,13.0
2,60,15709,2022-09-21,{12.5: 1},60,15709,2023-04-05,13.0
3,60,15709,2022-11-20,{7.0: 1},60,15709,2023-04-05,13.0
4,60,15709,2023-01-04,{13.0: 1},60,15709,2023-04-05,13.0
5,60,15709,2023-02-18,{5.0: 1},60,15709,2023-04-05,13.0
7,60,95672,2022-05-14,{8.0: 1},60,95672,2023-03-23,6.5
8,60,95672,2022-12-05,{6.5: 1},60,95672,2023-03-23,6.5
9,60,95672,2022-12-22,{6.5: 1},60,95672,2023-03-23,6.5
10,60,95672,2023-01-08,{6.5: 1},60,95672,2023-03-23,6.5
11,60,95672,2023-02-03,{6.5: 1},60,95672,2023-03-23,6.5


In [ ]:
## apply removal of label to queries and filters. 

In [64]:
size_query_per_user_date_categ_groupby_rml= filter_src_dt_before_label( queries_per_user_date_categ_groupby,
                                                                       label_trx[["user_id","leaf_id","CREATED_DT" ,"transactions_mode"]],
                                                                       'session_start_dt','CREATED_DT','transactions_mode',
                               left_on_cols=["user_id","dominant_category_id"] , right_on_cols=[ "user_id","leaf_id"]  )

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [65]:
size_query_per_user_date_categ_groupby_rml.head(5)

,dominant_category_id,user_id,date,sizes,leaf_id,label_trx_dt,label_trx_size
26,15709,14355,2023-02-17,{10.5: 1},15709,2023-03-25,10.5
33,15709,18346,2023-02-05,{11.0: 1},15709,2023-02-10,11.0
34,15709,18346,2023-02-09,{11.0: 1},15709,2023-02-10,11.0
35,15709,19016,2023-03-05,{10.0: 1},15709,2023-03-24,10.0
39,15709,22369,2023-03-20,{11.0: 1},15709,2023-04-12,14.0


In [66]:
size_filter_per_user_date_categ_groupby_rml= filter_src_dt_before_label( filters_per_user_date_categ_groupby,
                                                                       label_trx[["user_id","leaf_id","CREATED_DT","transactions_mode" ]],
                                                                       'session_start_dt','CREATED_DT',"transactions_mode",
                               left_on_cols=["user_id","dominant_category_id"] , right_on_cols=[ "user_id","leaf_id"]  )

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [67]:
size_filter_per_user_date_categ_groupby_rml[size_filter_per_user_date_categ_groupby_rml['user_id']==5242] 


,dominant_category_id,user_id,date,sizes,leaf_id,label_trx_dt,label_trx_size
25,15709,5242,2023-02-10,{10.0: 1},15709,2023-02-16,10.0


### Merge transactions, queries, searches: 

In [68]:
size_query_per_user_date_categ_groupby_rml.head(2)

,dominant_category_id,user_id,date,sizes,leaf_id,label_trx_dt,label_trx_size
26,15709,14355,2023-02-17,{10.5: 1},15709,2023-03-25,10.5
33,15709,18346,2023-02-05,{11.0: 1},15709,2023-02-10,11.0


In [69]:
size_filter_per_user_date_categ_groupby_rml.head(2)

,dominant_category_id,user_id,date,sizes,leaf_id,label_trx_dt,label_trx_size
25,15709,5242,2023-02-10,{10.0: 1},15709,2023-02-16,10.0
30,15709,6201,2023-02-24,{9.5: 1},15709,2023-02-25,9.5


In [70]:
trx_per_user_date_categ_groupby_rml.head(2)

,BUYER_ID,LEAF_CATEG_ID,date,sizes,user_id,leaf_id,label_trx_dt,label_trx_size
1,60,15709,2022-05-15,{12.0: 1},60,15709,2023-04-05,13.0
2,60,15709,2022-09-21,{12.5: 1},60,15709,2023-04-05,13.0


In [71]:
def src_aligned_cols(df_src: pd.DataFrame,   
                               user_id_col: str = 'user_id',
                               leaf_id_col='leaf_category_id',
                               dt_col:str='date', size_col:str='sizes',label_trx_dt_col:str= 'label_trx_dt',
                               label_trx_size_col:str= 'label_trx_size' ):
    
    history_type= pd.DataFrame({"user_id": df_src[user_id_col].astype(int), \
                                "leaf_id":df_src[leaf_id_col].astype(int),\
                                "date": df_src[dt_col],\
                                "sizes": df_src[size_col], 
                                "label_trx_dt": df_src[label_trx_dt_col],
                                "label_trx_size": df_src[label_trx_size_col]}) 
    return history_type
    

In [72]:
queries=src_aligned_cols(size_query_per_user_date_categ_groupby_rml, 
                        user_id_col='user_id' ,leaf_id_col='dominant_category_id',dt_col='date', size_col='sizes' ,label_trx_dt_col= 'label_trx_dt', label_trx_size_col= 'label_trx_size')  


In [73]:
filters=src_aligned_cols(size_filter_per_user_date_categ_groupby_rml,
                        user_id_col='user_id' ,leaf_id_col='dominant_category_id',dt_col='date', size_col='sizes',label_trx_dt_col= 'label_trx_dt', label_trx_size_col= 'label_trx_size' )  


In [74]:
transactions =src_aligned_cols (trx_per_user_date_categ_groupby_rml, 
                        user_id_col='BUYER_ID' ,leaf_id_col='LEAF_CATEG_ID',dt_col='date', size_col='sizes',label_trx_dt_col= 'label_trx_dt', label_trx_size_col= 'label_trx_size' )  

In [75]:
queries.head(5)

,user_id,leaf_id,date,sizes,label_trx_dt,label_trx_size
26,14355,15709,2023-02-17,{10.5: 1},2023-03-25,10.5
33,18346,15709,2023-02-05,{11.0: 1},2023-02-10,11.0
34,18346,15709,2023-02-09,{11.0: 1},2023-02-10,11.0
35,19016,15709,2023-03-05,{10.0: 1},2023-03-24,10.0
39,22369,15709,2023-03-20,{11.0: 1},2023-04-12,14.0


#### save 3 sources after leakage removal, in granularity of: user, leaf, date, sizes, label date, label trx size:

In [76]:
transactions.to_pickle('/data/ebay/data/olivyatan/size_data/trx_rml_label_perdate_categ_user_030723.pkl')
queries.to_pickle('/data/ebay/data/olivyatan/size_data/queries_rml_label_perdate_categ_userr_030723.pkl')
filters.to_pickle('/data/ebay/data/olivyatan/size_data/filters_rml_label_perdate_categ_userr_030723.pkl')
label_trx.to_pickle('/data/ebay/data/olivyatan/size_data/label_rml_label_perdate_categ_userr_030723.pkl')

### Prepare featues of basic model stats aggregated by all 3 sources:

#### Merge signals: modified aggregation to be per leaf and user:

In [ ]:
#merge 3 sources of trx, queries, filters size per user and categ: 

In [77]:
def merge_signals(df1: pd.DataFrame, df2: pd.DataFrame, df3=None):
    if df3 is not None:
        df = pd.concat([df1[['user_id','leaf_id', 'sizes']], df2[['user_id', 'leaf_id','sizes']], df3[['user_id', 'leaf_id', 'sizes']]])
    else:
        df = pd.concat([df1[['user_id',  'leaf_id','sizes']], df2[['user_id', 'leaf_id', 'sizes']]])

    df_merged = df.groupby(['user_id','leaf_id']).apply(lambda x: merge_counter_sizes(x['sizes'])).reset_index()
    df_merged.columns = ['user_id', 'leaf_id','sizes']

    print(f"DataFrame shape following merging sizes counts: {df_merged.shape}")

    df_agg = agg_sizes(df_merged, 'sizes', col_prefix='merged')

    return df_agg

##### user_id 60 :

In [100]:
label_trx[label_trx['user_id']==60]

,user_id,leaf_id,CREATED_DT,sizes,transactions_std,transactions_mean,transactions_median,transactions_mode,transactions_maxMinDiff,transactions_countOfEvidence,row_number
6,60,15709,2023-04-05,{13.0: 1},0.0,13.0,13.0,13.0,0.0,1,1
14,60,95672,2023-03-23,{6.5: 1},0.0,6.5,6.5,6.5,0.0,1,1


In [144]:
#men_merged_signals = 
m=merge_signals(queries[queries['user_id']==60] , filters[filters['user_id']==60], transactions[transactions['user_id']==60])

DataFrame shape following merging sizes counts: (2, 3)


In [145]:
m

,user_id,leaf_id,sizes,merged_std,merged_mean,merged_median,merged_mode,merged_maxMinDiff,merged_countOfEvidence
0,60,15709,"{12.0: 1, 12.5: 1, 7.0: 1, 13.0: 1, 5.0: 1}",3.261901,9.900000,12.0,5.0,8.0,5
1,60,95672,"{6.5: 28, 8.0: 1, 8.5: 1}",0.441273,6.616667,6.5,6.5,2.0,30


#### Add label to merged signals: 

In [82]:
merged_signals=merge_signals(queries , filters, transactions)

DataFrame shape following merging sizes counts: (1035021, 3)


In [83]:
merge_label=pd.merge(merged_signals, label_trx, on=["user_id","leaf_id"])

In [84]:
merge_label.rename(columns={"sizes_x":"sizes","sizes_y": "label_user_size","transactions_median":"label_trx_size" }, inplace=True)

In [85]:
merge_label.drop(columns=['transactions_std', 'transactions_mode','transactions_mean','transactions_maxMinDiff','row_number'],inplace=True)

#### df of basic features - input for model training :

In [86]:
merge_label.head(6)

,user_id,leaf_id,sizes,merged_std,merged_mean,merged_median,merged_mode,merged_maxMinDiff,merged_countOfEvidence,CREATED_DT,label_user_size,label_trx_size,transactions_countOfEvidence
0,60,15709,"{12.0: 1, 12.5: 1, 7.0: 1, 13.0: 1, 5.0: 1}",3.261901,9.900000,12.0,5.0,8.0,5,2023-04-05,{13.0: 1},13.0,1
1,60,95672,"{6.5: 28, 8.0: 1, 8.5: 1}",0.441273,6.616667,6.5,6.5,2.0,30,2023-03-23,{6.5: 1},6.5,1
2,77,95672,{8.5: 2},0.000000,8.500000,8.5,8.5,0.0,2,2022-10-23,{8.5: 1},8.5,1
3,192,95672,{8.5: 1},0.000000,8.500000,8.5,8.5,0.0,1,2022-06-24,{8.5: 1},8.5,1
4,2520,15709,{12.0: 1},0.000000,12.000000,12.0,12.0,0.0,1,2023-01-07,{13.0: 1},13.0,1
5,2642,15709,"{11.0: 1, 10.5: 3}",0.216506,10.625000,10.5,10.5,0.5,4,2022-10-25,{10.5: 1},10.5,1


In [87]:
merge_label.shape

(1035021, 13)

In [88]:
merge_label['label_trx_size'].value_counts()
#validate that size distribute uniformly. 

10.00    118805
11.00    108444
9.00     106145
10.50     89329
12.00     86611
9.50      85798
8.00      79306
8.50      69333
13.00     59610
11.50     50820
7.00      47512
7.50      37494
6.00      21242
14.00     19716
6.50      18778
12.50      9764
5.00       7784
15.00      7446
5.50       6286
16.00      2019
4.50       1569
13.50       928
14.50       232
15.50        47
10.25         1
9.75          1
10.75         1
Name: label_trx_size, dtype: int64

#### Save training data for a model of the basic stats of the pipepline: 

In [793]:
# pkl keeps counter format in read write 

In [89]:
merge_label.to_pickle('/data/ebay/data/olivyatan/size_data/merge_label_4features_per_userCateg_03072023.pkl')

In [90]:
merge_label.head(1)

,user_id,leaf_id,sizes,merged_std,merged_mean,merged_median,merged_mode,merged_maxMinDiff,merged_countOfEvidence,CREATED_DT,label_user_size,label_trx_size,transactions_countOfEvidence
0,60,15709,"{12.0: 1, 12.5: 1, 7.0: 1, 13.0: 1, 5.0: 1}",3.261901,9.9,12.0,5.0,8.0,5,2023-04-05,{13.0: 1},13.0,1


In [ ]:
## add data of size mode/median per user per recall source: for data 

### Add features per source of stats and time: 

#### Add features of aggregated stats :meidan/mode/mean/num_evidence_std per source, user, and category:

####  Add agg stats for flters, searches, queries, filters per user and categ: 

In [115]:
filters[filters['user_id']==5242]

,user_id,leaf_id,date,sizes,label_trx_dt,label_trx_size
25,5242,15709,2023-02-10,{10.0: 1},2023-02-16,10.0


In [116]:
queries[queries['user_id']==5242]

,user_id,leaf_id,date,sizes,label_trx_dt,label_trx_size


In [117]:
transactions[transactions['user_id']==5242]

,user_id,leaf_id,date,sizes,label_trx_dt,label_trx_size


### Per source add in granularity of user and categ: agg stats - merge /mode /mean /evidence/ std /maxmindiff

In [91]:
def merge_history_src_signals_user_categ(df1: pd.DataFrame,sizes_col: str,prefix:str ):
   
    df_merged = df1.groupby(['user_id','leaf_id']).apply(lambda x: merge_counter_sizes(x[sizes_col])).reset_index()
    df_merged.columns = ['user_id', 'leaf_id', sizes_col]

    print(f"DataFrame shape following merging sizes counts: {df_merged.shape}")

    df_agg = agg_sizes(df_merged, sizes_col, col_prefix=prefix)

    return df_agg

In [92]:
def agg_sizes(input_df: pd.DataFrame, sizes_col: str, col_prefix: str) -> pd.DataFrame:
    df = input_df.copy()

    df[f'{col_prefix}_std'] = [np.std(list(i.elements())) for i in df[sizes_col]]
    
    df[f'{col_prefix}_mean'] = [np.mean(list(i.elements())) for i in df[sizes_col]]
    
    df[f'{col_prefix}_median'] = [np.median(list(i.elements())) for i in df[sizes_col]]
    
    df[f'{col_prefix}_mode'] = [mode(list(i.elements())) for i in df[sizes_col]]

    df[f'{col_prefix}_maxMinDiff'] = calc_max_min_diff(df[sizes_col])

    df[f'{col_prefix}_countOfEvidence'] = [sum(i.values()) for i in df[sizes_col]]

    return df

In [93]:
 df_merge_queries_per_user_categ=merge_history_src_signals_user_categ(queries, 'sizes','queries')

DataFrame shape following merging sizes counts: (208013, 3)


In [94]:
 df_merge_filters_per_user_categ=merge_history_src_signals_user_categ(filters, 'sizes','filters')

DataFrame shape following merging sizes counts: (235650, 3)


In [95]:
 df_merge_transactions_per_user_categ=merge_history_src_signals_user_categ(transactions, 'sizes','transactions')

DataFrame shape following merging sizes counts: (909456, 3)


In [278]:
df = filters[filters['user_id']==60].groupby(['user_id','leaf_id']).apply(lambda x: merge_counter_sizes(x['sizes'])).reset_index()
df.columns = ['user_id', 'leaf_id', 'sizes']
df

,user_id,leaf_id,sizes
0,60,15709,"{12.0: 1, 12.5: 1, 7.0: 1, 13.0: 1, 5.0: 1}"
1,60,95672,"{8.0: 1, 6.5: 28, 8.5: 1}"


In [300]:
df[f'filters_std'] = [np.median(list(i.elements())) for i in df['sizes']]

In [288]:
df_agg = agg_sizes(df, 'sizes', col_prefix='filters')

0    3.261901
1    0.441273
Name: filters_std, dtype: float64
0    9.900000
1    6.616667
Name: filters_mean, dtype: float64
0    12.0
1     6.5
Name: filters_median, dtype: float64


In [280]:
df_agg

,user_id,leaf_id,sizes,filters_std,filters_mean,filters_median,filters_mode,filters_maxMinDiff,filters_countOfEvidence
0,60,15709,"{12.0: 1, 12.5: 1, 7.0: 1, 13.0: 1, 5.0: 1}",3.261901,9.900000,12.0,5.0,8.0,5
1,60,95672,"{8.0: 1, 6.5: 28, 8.5: 1}",0.441273,6.616667,6.5,6.5,2.0,30


In [96]:
df_merge_filters_per_user_categ[df_merge_filters_per_user_categ['user_id']==60]

,user_id,leaf_id,sizes,filters_std,filters_mean,filters_median,filters_mode,filters_maxMinDiff,filters_countOfEvidence
0,60,95672,{6.5: 18},0.0,6.5,6.5,6.5,0.0,18


In [127]:
filters[filters['user_id']==60]

,user_id,leaf_id,date,sizes,label_trx_dt,label_trx_size
5791546,60,95672,2023-02-15,{6.5: 6},2023-03-23,6.5
5791547,60,95672,2023-02-16,{6.5: 1},2023-03-23,6.5
5791548,60,95672,2023-02-23,{6.5: 1},2023-03-23,6.5
5791549,60,95672,2023-02-24,{6.5: 2},2023-03-23,6.5
5791550,60,95672,2023-02-26,{6.5: 2},2023-03-23,6.5
5791551,60,95672,2023-02-27,{6.5: 1},2023-03-23,6.5
5791552,60,95672,2023-03-01,{6.5: 5},2023-03-23,6.5


In [97]:
df_merge_queries_per_user_categ[df_merge_queries_per_user_categ['user_id']==60]

,user_id,leaf_id,sizes,queries_std,queries_mean,queries_median,queries_mode,queries_maxMinDiff,queries_countOfEvidence
0,60,95672,{6.5: 5},0.0,6.5,6.5,6.5,0.0,5


In [98]:
queries[queries['user_id']==60]

,user_id,leaf_id,date,sizes,label_trx_dt,label_trx_size
3328645,60,95672,2023-03-09,{6.5: 5},2023-03-23,6.5


In [99]:
df_merge_transactions_per_user_categ[df_merge_transactions_per_user_categ['user_id']==60]

,user_id,leaf_id,sizes,transactions_std,transactions_mean,transactions_median,transactions_mode,transactions_maxMinDiff,transactions_countOfEvidence
0,60,15709,"{12.0: 1, 12.5: 1, 7.0: 1, 13.0: 1, 5.0: 1}",3.261901,9.9,12.0,5.0,8.0,5
1,60,95672,"{8.0: 1, 6.5: 5, 8.5: 1}",0.801784,7.0,6.5,6.5,2.0,7


In [100]:
transactions[transactions['user_id']==60]

,user_id,leaf_id,date,sizes,label_trx_dt,label_trx_size
1,60,15709,2022-05-15,{12.0: 1},2023-04-05,13.0
2,60,15709,2022-09-21,{12.5: 1},2023-04-05,13.0
3,60,15709,2022-11-20,{7.0: 1},2023-04-05,13.0
4,60,15709,2023-01-04,{13.0: 1},2023-04-05,13.0
5,60,15709,2023-02-18,{5.0: 1},2023-04-05,13.0
7,60,95672,2022-05-14,{8.0: 1},2023-03-23,6.5
8,60,95672,2022-12-05,{6.5: 1},2023-03-23,6.5
9,60,95672,2022-12-22,{6.5: 1},2023-03-23,6.5
10,60,95672,2023-01-08,{6.5: 1},2023-03-23,6.5
11,60,95672,2023-02-03,{6.5: 1},2023-03-23,6.5


In [101]:
df_merge_filters_per_user_categ[df_merge_filters_per_user_categ['user_id']==60]

,user_id,leaf_id,sizes,filters_std,filters_mean,filters_median,filters_mode,filters_maxMinDiff,filters_countOfEvidence
0,60,95672,{6.5: 18},0.0,6.5,6.5,6.5,0.0,18


In [102]:
size_filter_per_user_date_categ_groupby_rml[size_filter_per_user_date_categ_groupby_rml['user_id']==60]

,dominant_category_id,user_id,date,sizes,leaf_id,label_trx_dt,label_trx_size
5791546,95672,60,2023-02-15,{6.5: 6},95672,2023-03-23,6.5
5791547,95672,60,2023-02-16,{6.5: 1},95672,2023-03-23,6.5
5791548,95672,60,2023-02-23,{6.5: 1},95672,2023-03-23,6.5
5791549,95672,60,2023-02-24,{6.5: 2},95672,2023-03-23,6.5
5791550,95672,60,2023-02-26,{6.5: 2},95672,2023-03-23,6.5
5791551,95672,60,2023-02-27,{6.5: 1},95672,2023-03-23,6.5
5791552,95672,60,2023-03-01,{6.5: 5},95672,2023-03-23,6.5


In [113]:
filters[filters['user_id']==77]

,user_id,leaf_id,date,sizes,label_trx_dt,label_trx_size


In [114]:
transactions[transactions['user_id']==77]

,user_id,leaf_id,date,sizes,label_trx_dt,label_trx_size
15,77,95672,2022-04-28,{8.5: 1},2022-10-23,8.5
16,77,95672,2022-10-16,{8.5: 1},2022-10-23,8.5


In [104]:
df_merge_filters_per_user_categ[df_merge_filters_per_user_categ['user_id']==77]

,user_id,leaf_id,sizes,filters_std,filters_mean,filters_median,filters_mode,filters_maxMinDiff,filters_countOfEvidence


In [111]:
df_merge_queries_per_user_categ[df_merge_queries_per_user_categ['user_id']==77]

,user_id,leaf_id,sizes,queries_std,queries_mean,queries_median,queries_mode,queries_maxMinDiff,queries_countOfEvidence


In [112]:
df_merge_transactions_per_user_categ[df_merge_transactions_per_user_categ['user_id']==77]

,user_id,leaf_id,sizes,transactions_std,transactions_mean,transactions_median,transactions_mode,transactions_maxMinDiff,transactions_countOfEvidence
2,77,95672,{8.5: 2},0.0,8.5,8.5,8.5,0.0,2


In [105]:
df_merge_transactions_per_user_categ.to_pickle('/data/ebay/data/olivyatan/size_data/merge_stat_trx_per_categ_user_03072023.pkl')
df_merge_queries_per_user_categ.to_pickle('/data/ebay/data/olivyatan/size_data/merge_stat_queries_per_categ_user_03072023.pkl')
df_merge_filters_per_user_categ.to_pickle('/data/ebay/data/olivyatan/size_data/merge_stat_filters_per_categ_user_03072023.pkl')

### Add features per src with stats of above to aggregated features, and time features: 

# Add features of date. 



In [115]:
transactions.head(2)

,user_id,leaf_id,date,sizes,label_trx_dt,label_trx_size
1,60,15709,2022-05-15,{12.0: 1},2023-04-05,13.0
2,60,15709,2022-09-21,{12.5: 1},2023-04-05,13.0


In [116]:
df_merge_queries_per_user_categ[df_merge_queries_per_user_categ['user_id']==60] 

,user_id,leaf_id,sizes,queries_std,queries_mean,queries_median,queries_mode,queries_maxMinDiff,queries_countOfEvidence
0,60,95672,{6.5: 5},0.0,6.5,6.5,6.5,0.0,5


In [126]:
df_merge_filters_per_user_categ[df_merge_filters_per_user_categ['user_id']==60] 

,user_id,leaf_id,sizes,filters_std,filters_mean,filters_median,filters_mode,filters_maxMinDiff,filters_countOfEvidence
0,60,95672,{6.5: 18},0.0,6.5,6.5,6.5,0.0,18


In [118]:
df_merge_transactions_per_user_categ[df_merge_transactions_per_user_categ['user_id']==60] 

,user_id,leaf_id,sizes,transactions_std,transactions_mean,transactions_median,transactions_mode,transactions_maxMinDiff,transactions_countOfEvidence
0,60,15709,"{12.0: 1, 12.5: 1, 7.0: 1, 13.0: 1, 5.0: 1}",3.261901,9.9,12.0,5.0,8.0,5
1,60,95672,"{8.0: 1, 6.5: 5, 8.5: 1}",0.801784,7.0,6.5,6.5,2.0,7


### Features of history type #6 stats , Merge 3 sources aggregated stats with label: 

In [119]:
merged_label_srcs=pd.merge(merge_label,df_merge_transactions_per_user_categ,  on=['user_id','leaf_id'], how='left').rename(columns={'sizes_x': 'merged_sizes_all_srcs'})
merged_label_srcs = pd.merge(merged_label_srcs, df_merge_queries_per_user_categ,  on=['user_id','leaf_id'], how='left')
merged_label_srcs = pd.merge(merged_label_srcs, df_merge_filters_per_user_categ, on=['user_id','leaf_id'], how='left')

In [120]:
merged_label_srcs.columns

Index(['user_id', 'leaf_id', 'merged_sizes_all_srcs', 'merged_std',
       'merged_mean', 'merged_median', 'merged_mode', 'merged_maxMinDiff',
       'merged_countOfEvidence', 'CREATED_DT', 'label_user_size',
       'label_trx_size', 'transactions_countOfEvidence_x', 'sizes_y',
       'transactions_std', 'transactions_mean', 'transactions_median',
       'transactions_mode', 'transactions_maxMinDiff',
       'transactions_countOfEvidence_y', 'sizes_x', 'queries_std',
       'queries_mean', 'queries_median', 'queries_mode', 'queries_maxMinDiff',
       'queries_countOfEvidence', 'sizes_y', 'filters_std', 'filters_mean',
       'filters_median', 'filters_mode', 'filters_maxMinDiff',
       'filters_countOfEvidence'],
      dtype='object')

In [121]:
merge_label.head(1)

,user_id,leaf_id,sizes,merged_std,merged_mean,merged_median,merged_mode,merged_maxMinDiff,merged_countOfEvidence,CREATED_DT,label_user_size,label_trx_size,transactions_countOfEvidence
0,60,15709,"{12.0: 1, 12.5: 1, 7.0: 1, 13.0: 1, 5.0: 1}",3.261901,9.9,12.0,5.0,8.0,5,2023-04-05,{13.0: 1},13.0,1


In [122]:
# Rename and change column order
new_columns = { 'CREATED_DT': 'label_trx_dt','transactions_countOfEvidence_y':'transactions_countOfEvidence'}
merged_label_srcs = merged_label_srcs.rename(columns=new_columns)[['user_id', 'leaf_id', 'merged_sizes_all_srcs', 'merged_std', 'merged_mean',
       'merged_median', 'merged_mode', 'merged_maxMinDiff',
       'merged_countOfEvidence', 
       'transactions_std', 'transactions_mean', 'transactions_median',
       'transactions_mode', 'transactions_maxMinDiff',
       'transactions_countOfEvidence', 'queries_std',
       'queries_mean', 'queries_median', 'queries_mode', 'queries_maxMinDiff',
       'queries_countOfEvidence', 'filters_std', 'filters_mean',
       'filters_median', 'filters_mode', 'filters_maxMinDiff',
       'filters_countOfEvidence','label_trx_dt', 'label_user_size',
       'label_trx_size' ]]

In [123]:
merged_label_srcs.columns

Index(['user_id', 'leaf_id', 'merged_sizes_all_srcs', 'merged_std',
       'merged_mean', 'merged_median', 'merged_mode', 'merged_maxMinDiff',
       'merged_countOfEvidence', 'transactions_std', 'transactions_mean',
       'transactions_median', 'transactions_mode', 'transactions_maxMinDiff',
       'transactions_countOfEvidence', 'queries_std', 'queries_mean',
       'queries_median', 'queries_mode', 'queries_maxMinDiff',
       'queries_countOfEvidence', 'filters_std', 'filters_mean',
       'filters_median', 'filters_mode', 'filters_maxMinDiff',
       'filters_countOfEvidence', 'label_trx_dt', 'label_user_size',
       'label_trx_size'],
      dtype='object')

In [124]:
merged_label_srcs.head(3)

,user_id,leaf_id,merged_sizes_all_srcs,merged_std,merged_mean,merged_median,merged_mode,merged_maxMinDiff,merged_countOfEvidence,transactions_std,...,queries_countOfEvidence,filters_std,filters_mean,filters_median,filters_mode,filters_maxMinDiff,filters_countOfEvidence,label_trx_dt,label_user_size,label_trx_size
0,60,15709,"{12.0: 1, 12.5: 1, 7.0: 1, 13.0: 1, 5.0: 1}",3.261901,9.900000,12.0,5.0,8.0,5,3.261901,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-05,{13.0: 1},13.0
1,60,95672,"{6.5: 28, 8.0: 1, 8.5: 1}",0.441273,6.616667,6.5,6.5,2.0,30,0.801784,...,5.0,0.0,6.5,6.5,6.5,0.0,18.0,2023-03-23,{6.5: 1},6.5
2,77,95672,{8.5: 2},0.000000,8.500000,8.5,8.5,0.0,2,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-23,{8.5: 1},8.5


In [125]:
print(merged_label_srcs[merged_label_srcs['user_id']==60] )


  user_id leaf_id                        merged_sizes_all_srcs  merged_std  \
0      60   15709  {12.0: 1, 12.5: 1, 7.0: 1, 13.0: 1, 5.0: 1}    3.261901   
1      60   95672                    {6.5: 28, 8.0: 1, 8.5: 1}    0.441273   

   merged_mean  merged_median  merged_mode  merged_maxMinDiff  \
0     9.900000           12.0          5.0                8.0   
1     6.616667            6.5          6.5                2.0   

   merged_countOfEvidence  transactions_std  ...  queries_countOfEvidence  \
0                       5          3.261901  ...                      NaN   
1                      30          0.801784  ...                      5.0   

   filters_std  filters_mean  filters_median  filters_mode  \
0          NaN           NaN             NaN           NaN   
1          0.0           6.5             6.5           6.5   

   filters_maxMinDiff  filters_countOfEvidence  label_trx_dt  label_user_size  \
0                 NaN                      NaN    2023-04-05        {

In [128]:
merged_label_srcs.loc[4, 'queries_countOfEvidence']

nan

#### Final features data saving:

In [194]:
merged_label_srcs.to_pickle('/data/ebay/data/olivyatan/size_data/merged_label_srcs_stats_persrc_040723.pkl')

In [130]:
merged_label_srcs[merged_label_srcs['user_id']==5242] ['filters_std'].head(7)

13    0.0
Name: filters_std, dtype: float64

In [131]:
merged_label_srcs[merged_label_srcs['user_id']==77]['filters_std'].head(7)

2   NaN
Name: filters_std, dtype: float64

In [133]:
merged_label_srcs[merged_label_srcs['user_id']==77]

,user_id,leaf_id,merged_sizes_all_srcs,merged_std,merged_mean,merged_median,merged_mode,merged_maxMinDiff,merged_countOfEvidence,transactions_std,...,queries_countOfEvidence,filters_std,filters_mean,filters_median,filters_mode,filters_maxMinDiff,filters_countOfEvidence,label_trx_dt,label_user_size,label_trx_size
2,77,95672,{8.5: 2},0.0,8.5,8.5,8.5,0.0,2,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-23,{8.5: 1},8.5


In [134]:
filters[filters['user_id']==5242]

,user_id,leaf_id,date,sizes,label_trx_dt,label_trx_size
25,5242,15709,2023-02-10,{10.0: 1},2023-02-16,10.0


In [135]:
queries[queries['user_id']==5242]

,user_id,leaf_id,date,sizes,label_trx_dt,label_trx_size


In [136]:
transactions[transactions['user_id']==5242]

,user_id,leaf_id,date,sizes,label_trx_dt,label_trx_size


In [137]:
df_merge_filters_per_user_categ[df_merge_filters_per_user_categ['user_id']==2520]

,user_id,leaf_id,sizes,filters_std,filters_mean,filters_median,filters_mode,filters_maxMinDiff,filters_countOfEvidence


In [138]:
filters.shape, queries.shape, transactions.shape, label_trx.shape

((1477479, 6), (809763, 6), (2328873, 6), (3201768, 11))

In [139]:
filters.columns

Index(['user_id', 'leaf_id', 'date', 'sizes', 'label_trx_dt',
       'label_trx_size'],
      dtype='object')

In [140]:
# Calculate the percentage of non-null values for each column
percentage_not_na = merged_label_srcs.count() / len(merged_label_srcs) * 100

# Print the results
print(percentage_not_na)

user_id                         100.000000
leaf_id                         100.000000
merged_sizes_all_srcs           100.000000
merged_std                      100.000000
merged_mean                     100.000000
merged_median                   100.000000
merged_mode                     100.000000
merged_maxMinDiff               100.000000
merged_countOfEvidence          100.000000
transactions_std                 87.868362
transactions_mean                87.868362
transactions_median              87.868362
transactions_mode                87.868362
transactions_maxMinDiff          87.868362
transactions_countOfEvidence     87.868362
queries_std                      20.097467
queries_mean                     20.097467
queries_median                   20.097467
queries_mode                     20.097467
queries_maxMinDiff               20.097467
queries_countOfEvidence          20.097467
filters_std                      22.767654
filters_mean                     22.767654
filters_med

In [141]:
merged_label_srcs.head(3)

,user_id,leaf_id,merged_sizes_all_srcs,merged_std,merged_mean,merged_median,merged_mode,merged_maxMinDiff,merged_countOfEvidence,transactions_std,...,queries_countOfEvidence,filters_std,filters_mean,filters_median,filters_mode,filters_maxMinDiff,filters_countOfEvidence,label_trx_dt,label_user_size,label_trx_size
0,60,15709,"{12.0: 1, 12.5: 1, 7.0: 1, 13.0: 1, 5.0: 1}",3.261901,9.900000,12.0,5.0,8.0,5,3.261901,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-05,{13.0: 1},13.0
1,60,95672,"{6.5: 28, 8.0: 1, 8.5: 1}",0.441273,6.616667,6.5,6.5,2.0,30,0.801784,...,5.0,0.0,6.5,6.5,6.5,0.0,18.0,2023-03-23,{6.5: 1},6.5
2,77,95672,{8.5: 2},0.000000,8.500000,8.5,8.5,0.0,2,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-23,{8.5: 1},8.5


In [142]:
### add time features 

In [143]:
transactions[transactions['user_id']==77].head(3)

,user_id,leaf_id,date,sizes,label_trx_dt,label_trx_size
15,77,95672,2022-04-28,{8.5: 1},2022-10-23,8.5
16,77,95672,2022-10-16,{8.5: 1},2022-10-23,8.5


In [144]:
queries[queries['user_id']==77].head(3)

,user_id,leaf_id,date,sizes,label_trx_dt,label_trx_size


In [145]:
filters[filters['user_id']==77].head(3)

,user_id,leaf_id,date,sizes,label_trx_dt,label_trx_size


### Add features of time: 

In [146]:
filters.shape

(1477479, 6)

In [132]:
# Filter rows where the Counter dictionary has exactly two keys
f_df = filters[filters['sizes'].apply(lambda x: len(x.keys()) >= 2)]

In [147]:
 f = pd.concat([transactions[['user_id', 'leaf_id', 'sizes', 'date', 'label_trx_dt', 'label_trx_size']],
                        queries[['user_id', 'leaf_id', 'sizes', 'date', 'label_trx_dt', 'label_trx_size']],
                        filters[['user_id', 'leaf_id', 'sizes', 'date', 'label_trx_dt', 'label_trx_size']]])

In [148]:
#what is the amount of values in sizes: we need this to conclude on time features? 
counts = f['sizes'].apply(lambda x: len(x.keys())).value_counts()

# Calculate the total number of rows
total_rows = len(f)

# Initialize dictionaries to store the counts and percentages
counts_dict = {}
percentages_dict = {}

# Iterate over the counts and calculate the percentages
for i, count in counts.items():
    counts_dict[i] = count
    percentages_dict[i] = count / total_rows * 100

# Calculate the maximum number of keys
max_keys = max(list(counts_dict.keys()))

for i in range(1, max_keys + 1):
    print(f"Rows with {i} or more keys: {counts_dict.get(i, 0)} ({percentages_dict.get(i, 0):.2f}%)")
# top 2 are 93% of the cases. 

Rows with 1 or more keys: 3762594 (81.51%)
Rows with 2 or more keys: 545574 (11.82%)
Rows with 3 or more keys: 183357 (3.97%)
Rows with 4 or more keys: 64162 (1.39%)
Rows with 5 or more keys: 26767 (0.58%)
Rows with 6 or more keys: 12548 (0.27%)
Rows with 7 or more keys: 7083 (0.15%)
Rows with 8 or more keys: 3708 (0.08%)
Rows with 9 or more keys: 3057 (0.07%)
Rows with 10 or more keys: 1835 (0.04%)
Rows with 11 or more keys: 1400 (0.03%)
Rows with 12 or more keys: 1162 (0.03%)
Rows with 13 or more keys: 952 (0.02%)
Rows with 14 or more keys: 532 (0.01%)
Rows with 15 or more keys: 426 (0.01%)
Rows with 16 or more keys: 336 (0.01%)
Rows with 17 or more keys: 255 (0.01%)
Rows with 18 or more keys: 45 (0.00%)
Rows with 19 or more keys: 157 (0.00%)
Rows with 20 or more keys: 121 (0.00%)
Rows with 21 or more keys: 28 (0.00%)
Rows with 22 or more keys: 8 (0.00%)
Rows with 23 or more keys: 4 (0.00%)
Rows with 24 or more keys: 4 (0.00%)


### Calculate feature of time for most common user most recent size :
#### User most recent size 
#### Most recent size days (comparing to trx date) : 1 , 2, 3, 4,5, 60 , 70 

##### most recent common size and days diff agregated for all sources:

In [149]:
import pandas as pd
import numpy as np
#most recent common size we in case of 2 sources have same most recent size date we take one of them(concat removes duplicates) 
def merge_signals_date_mostRecentSize_diff(df1: pd.DataFrame, df2: pd.DataFrame, df3=None, df1_history_type='queries', df2_history_type='filters', df3_history_type='transactions'):
    if df3 is not None:
        
        df = pd.concat([df1[['user_id', 'leaf_id', 'sizes', 'date', 'label_trx_dt', 'label_trx_size']],
                        df2[['user_id', 'leaf_id', 'sizes', 'date', 'label_trx_dt', 'label_trx_size']],
                        df3[['user_id', 'leaf_id', 'sizes', 'date', 'label_trx_dt', 'label_trx_size']]])
    else:
        df = pd.concat([df1[['user_id', 'leaf_id', 'sizes', 'date', 'label_trx_dt', 'label_trx_size']],
                        df2[['user_id', 'leaf_id', 'sizes', 'date', 'label_trx_dt', 'label_trx_size']]])

    df.reset_index(drop=True, inplace=True)  # Reset the index to avoid duplicates
     #most recent size:
    df['Rank_dt'] = df.sort_values(by=['user_id', 'leaf_id', 'date'], ascending=[True, True, False]).groupby(
        ['user_id', 'leaf_id']).cumcount() + 1

    most_recent_dt = df[df['Rank_dt'] == 1]
    most_recent_dt['mostRecentCommonSize_DaysDiff'] = (most_recent_dt['label_trx_dt'] - most_recent_dt['date']).dt.days
 
    # Iterate through each row and get the key-value pair with the maximum value in the sizes dictionary
    for index, row in most_recent_dt.iterrows():
        sizes = row['sizes']
  
        if sizes:
            max_size = max(sizes, key=sizes.get)
           
            most_recent_dt.at[index, 'mostRecentCommonSize'] = max_size
            most_recent_dt.at[index, 'mostRecentCommonSize_CountOfEvidence'] = sizes[max_size]

    # Fill None values with NaN
    most_recent_dt.fillna(np.nan, inplace=True)

    return most_recent_dt


In [ ]:
recent_size=merge_signals_date_mostRecentSize_diff(queries[queries['user_id']==60],filters[filters['user_id']==60], 
                                                   transactions[transactions['user_id']==60])


In [151]:
recent_size

,user_id,leaf_id,sizes,date,label_trx_dt,label_trx_size,Rank_dt,mostRecentCommonSize_DaysDiff,mostRecentCommonSize,mostRecentCommonSize_CountOfEvidence
12,60,15709,{5.0: 1},2023-02-18,2023-04-05,13.0,1,46,5.0,1.0
19,60,95672,{6.5: 1},2023-03-15,2023-03-23,6.5,1,8,6.5,1.0


In [152]:
most_recent_size_diff=merge_signals_date_mostRecentSize_diff(queries,  filters, transactions)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [153]:
most_recent_size_diff.head(10) 

,user_id,leaf_id,sizes,date,label_trx_dt,label_trx_size,Rank_dt,mostRecentCommonSize_DaysDiff,mostRecentCommonSize,mostRecentCommonSize_CountOfEvidence
0,14355,15709,{10.5: 1},2023-02-17,2023-03-25,10.5,1,36,10.5,1.0
2,18346,15709,{11.0: 1},2023-02-09,2023-02-10,11.0,1,1,11.0,1.0
3,19016,15709,{10.0: 1},2023-03-05,2023-03-24,10.0,1,19,10.0,1.0
6,23437,15709,{9.5: 1},2023-02-23,2023-03-01,9.5,1,6,9.5,1.0
9,24200,15709,{11.0: 5},2023-01-17,2023-01-18,11.5,1,1,11.0,5.0
11,25134,15709,{10.0: 1},2023-01-23,2023-01-28,10.5,1,5,10.0,1.0
12,30046,15709,{13.0: 1},2023-02-09,2023-02-12,6.5,1,3,13.0,1.0
13,30261,15709,{15.0: 3},2023-04-07,2023-04-08,15.0,1,1,15.0,3.0
15,39782,15709,{10.5: 1},2023-03-12,2023-03-22,7.5,1,10,10.5,1.0
16,44141,15709,{13.0: 1},2023-02-18,2023-03-09,13.0,1,19,13.0,1.0


#### Most recent common size and days diff , calculated per source : 

##### Examination on 2 users:

In [191]:
 transactions[transactions['user_id'].isin([2520,77])][['user_id', 'leaf_id', 'sizes', 'date', 'label_trx_dt', 'label_trx_size']]

,user_id,leaf_id,sizes,date,label_trx_dt,label_trx_size
15,77,95672,{8.5: 1},2022-04-28,2022-10-23,8.5
16,77,95672,{8.5: 1},2022-10-16,2022-10-23,8.5
39,2520,15709,{12.0: 1},2022-09-30,2023-01-07,13.0


In [192]:
queries[queries['user_id'].isin([2520,77])][['user_id', 'leaf_id', 'sizes', 'date', 'label_trx_dt', 'label_trx_size']]

,user_id,leaf_id,sizes,date,label_trx_dt,label_trx_size


In [193]:
filters[filters['user_id'].isin([2520,77])][['user_id', 'leaf_id', 'sizes', 'date', 'label_trx_dt', 'label_trx_size']]

,user_id,leaf_id,sizes,date,label_trx_dt,label_trx_size
15,77,95672,{8.5: 1},2022-04-28,2022-10-23,8.5
16,77,95672,{8.5: 1},2022-10-16,2022-10-23,8.5
39,2520,15709,{12.0: 1},2022-09-30,2023-01-07,13.0


In [194]:
new_column_values = ['queries', 'filters', 'transactions']
queries['history_type'] = new_column_values[0]
filters['history_type'] = new_column_values[1]
transactions['history_type'] = new_column_values[2]

df = pd.concat([queries[queries['user_id'].isin([77,2520])][['user_id', 'leaf_id', 'sizes', 'date', 'label_trx_dt', 'label_trx_size', 'history_type']],\
               filters[filters['user_id'].isin([77,2520])][['user_id', 'leaf_id', 'sizes', 'date', 'label_trx_dt', 'label_trx_size', 'history_type']],\
               transactions[transactions['user_id'].isin([77,2520])][['user_id', 'leaf_id', 'sizes', 'date', 'label_trx_dt', 'label_trx_size', 'history_type']]], ignore_index=False)

In [195]:
df

,user_id,leaf_id,sizes,date,label_trx_dt,label_trx_size,history_type
15,77,95672,{8.5: 1},2022-04-28,2022-10-23,8.5,filters
16,77,95672,{8.5: 1},2022-10-16,2022-10-23,8.5,filters
39,2520,15709,{12.0: 1},2022-09-30,2023-01-07,13.0,filters
15,77,95672,{8.5: 1},2022-04-28,2022-10-23,8.5,transactions
16,77,95672,{8.5: 1},2022-10-16,2022-10-23,8.5,transactions
39,2520,15709,{12.0: 1},2022-09-30,2023-01-07,13.0,transactions


In [196]:
df.reset_index( inplace=True) 
df['Rank_dt'] = df.sort_values(by=['user_id', 'leaf_id', 'history_type', 'date'], ascending=[True, True, True, False]).groupby(
        ['user_id', 'leaf_id', 'history_type']).cumcount() + 1

In [197]:
most_recent_dt = df[df['Rank_dt'] == 1]

In [198]:
most_recent_dt

,index,user_id,leaf_id,sizes,date,label_trx_dt,label_trx_size,history_type,Rank_dt
1,16,77,95672,{8.5: 1},2022-10-16,2022-10-23,8.5,filters,1
2,39,2520,15709,{12.0: 1},2022-09-30,2023-01-07,13.0,filters,1
4,16,77,95672,{8.5: 1},2022-10-16,2022-10-23,8.5,transactions,1
5,39,2520,15709,{12.0: 1},2022-09-30,2023-01-07,13.0,transactions,1


In [199]:

most_recent_dt['queries_mostRecentCommonSize_DaysDiff'] = np.where(most_recent_dt['history_type'] == 'queries', (most_recent_dt['label_trx_dt'] - most_recent_dt['date']).dt.days, np.nan)
most_recent_dt['filters_mostRecentCommonSize_DaysDiff'] = np.where(most_recent_dt['history_type'] == 'filters', (most_recent_dt['label_trx_dt'] - most_recent_dt['date']).dt.days, np.nan)
most_recent_dt['transactions_mostRecentCommonSize_DaysDiff'] = np.where(most_recent_dt['history_type'] == 'transactions', (most_recent_dt['label_trx_dt'] - most_recent_dt['date']).dt.days, np.nan)
 

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [200]:
most_recent_dt

,index,user_id,leaf_id,sizes,date,label_trx_dt,label_trx_size,history_type,Rank_dt,queries_mostRecentCommonSize_DaysDiff,filters_mostRecentCommonSize_DaysDiff,transactions_mostRecentCommonSize_DaysDiff
1,16,77,95672,{8.5: 1},2022-10-16,2022-10-23,8.5,filters,1,NaN,7.0,NaN
2,39,2520,15709,{12.0: 1},2022-09-30,2023-01-07,13.0,filters,1,NaN,99.0,NaN
4,16,77,95672,{8.5: 1},2022-10-16,2022-10-23,8.5,transactions,1,NaN,NaN,7.0
5,39,2520,15709,{12.0: 1},2022-09-30,2023-01-07,13.0,transactions,1,NaN,NaN,99.0


In [201]:
    for index, row in most_recent_dt.iterrows():
        
        sizes = row['sizes']
        if sizes:
            max_size = max(sizes, key=sizes.get)
            

            # Create additional columns for each previous column by history type
            for i in range(1, 4):
                
                if row['history_type'] == 'queries':
                    most_recent_dt.at[index, f'{row["history_type"]}_mostRecentCommonSize'] = max_size
                    most_recent_dt.at[index, f'{row["history_type"]}_mostRecentCommonSize_CountOfEvidence'] = sizes[max_size]
                    most_recent_dt.at[index, 'filters_mostRecentCommonSize'] = np.nan
                    most_recent_dt.at[index, 'filters_mostRecentCommonSize_CountOfEvidence'] = np.nan
                    most_recent_dt.at[index, 'transactions_mostRecentCommonSize'] = np.nan
                    most_recent_dt.at[index, 'transactions_mostRecentCommonSize_CountOfEvidence'] = np.nan
                elif row['history_type'] == 'filters':
                    most_recent_dt.at[index, f'{row["history_type"]}_mostRecentCommonSize'] = max_size
                    most_recent_dt.at[index, f'{row["history_type"]}_mostRecentCommonSize_CountOfEvidence'] = sizes[max_size]
                    most_recent_dt.at[index, 'queries_mostRecentCommonSize'] = np.nan
                    most_recent_dt.at[index, 'queries_mostRecentCommonSize_CountOfEvidence'] = np.nan
                    most_recent_dt.at[index, 'transactions_mostRecentCommonSize'] = np.nan
                    most_recent_dt.at[index, 'transactions_mostRecentCommonSize_CountOfEvidence'] = np.nan
                elif row['history_type'] == 'transactions':
                    most_recent_dt.at[index, f'{row["history_type"]}_mostRecentCommonSize'] = max_size
                    most_recent_dt.at[index, f'{row["history_type"]}_mostRecentCommonSize_CountOfEvidence'] = sizes[max_size]
                    most_recent_dt.at[index, 'queries_mostRecentCommonSize'] = np.nan
                    most_recent_dt.at[index, 'queries_mostRecentCommonSize_CountOfEvidence'] = np.nan
                    most_recent_dt.at[index, 'filters_mostRecentCommonSize'] = np.nan
                    most_recent_dt.at[index, 'filters_mostRecentCommonSize_CountOfEvidence'] = np.nan
    # Convert 'sizes' column from Counter to tuple
    most_recent_dt['sizes'] = most_recent_dt['sizes'].apply(lambda x: tuple(x.items()))

  
    # Group by the desired columns and keep the values after the max
    most_recent_dt = most_recent_dt.groupby(['user_id', 'leaf_id', 'label_trx_dt', 'label_trx_size']).agg({
        'queries_mostRecentCommonSize_DaysDiff': 'last',
        'filters_mostRecentCommonSize_DaysDiff': 'last',
        'transactions_mostRecentCommonSize_DaysDiff': 'last',
        
        'queries_mostRecentCommonSize': 'last',
        'filters_mostRecentCommonSize': 'last',
        'transactions_mostRecentCommonSize': 'last',
        
        'queries_mostRecentCommonSize_CountOfEvidence': 'last',
        'filters_mostRecentCommonSize_CountOfEvidence': 'last',
        'transactions_mostRecentCommonSize_CountOfEvidence': 'last'}) 

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [202]:
most_recent_dt

,,,,queries_mostRecentCommonSize_DaysDiff,filters_mostRecentCommonSize_DaysDiff,transactions_mostRecentCommonSize_DaysDiff,queries_mostRecentCommonSize,filters_mostRecentCommonSize,transactions_mostRecentCommonSize,queries_mostRecentCommonSize_CountOfEvidence,filters_mostRecentCommonSize_CountOfEvidence,transactions_mostRecentCommonSize_CountOfEvidence
user_id,leaf_id,label_trx_dt,label_trx_size,,,,,,,,,
77,95672,2022-10-23,8.5,NaN,7.0,7.0,NaN,8.5,8.5,NaN,1.0,1.0
2520,15709,2023-01-07,13.0,NaN,99.0,99.0,NaN,12.0,12.0,NaN,1.0,1.0


#### history_signals: mostRecentSize_diff features per source

In [154]:
def history_signals_date_mostRecentSize_diff(df1: pd.DataFrame, df2: pd.DataFrame, df3=None, df1_history_type='queries', df2_history_type='filters', df3_history_type='transactions'):
    
    # Add 'history_type' column to each dataframe with respective values
    df1['history_type'] = df1_history_type
    df2['history_type'] = df2_history_type
    
    if df3 is not None:
        df3['history_type'] = df3_history_type
        
     
        df = pd.concat([df1[['user_id', 'leaf_id', 'sizes', 'date', 'label_trx_dt', 'label_trx_size', 'history_type']],
                        df2[['user_id', 'leaf_id', 'sizes', 'date', 'label_trx_dt', 'label_trx_size', 'history_type']],
                        df3[['user_id', 'leaf_id', 'sizes', 'date', 'label_trx_dt', 'label_trx_size', 'history_type']]], ignore_index=False)
    else:
       
        df = pd.concat([df1[['user_id', 'leaf_id', 'sizes', 'date', 'label_trx_dt', 'label_trx_size', 'history_type']],
                        df2[['user_id', 'leaf_id', 'sizes', 'date', 'label_trx_dt', 'label_trx_size', 'history_type']]], ignore_index=False) #keep dup rows 

    df.reset_index( inplace=True)  
    #most recent size per source
    df['Rank_dt'] = df.sort_values(by=['user_id', 'leaf_id', 'history_type', 'date'], ascending=[True, True, True, False]).groupby(
        ['user_id', 'leaf_id', 'history_type']).cumcount() + 1

    most_recent_dt = df[df['Rank_dt'] == 1]
    # Update 'mostRecentCommonSize_DaysDiff' based on 'history_type'
    most_recent_dt['queries_mostRecentCommonSize_DaysDiff'] = np.where(most_recent_dt['history_type'] == 'queries', (most_recent_dt['label_trx_dt'] - most_recent_dt['date']).dt.days, np.nan)
    most_recent_dt['filters_mostRecentCommonSize_DaysDiff'] = np.where(most_recent_dt['history_type'] == 'filters', (most_recent_dt['label_trx_dt'] - most_recent_dt['date']).dt.days, np.nan)
    most_recent_dt['transactions_mostRecentCommonSize_DaysDiff'] = np.where(most_recent_dt['history_type'] == 'transactions', (most_recent_dt['label_trx_dt'] - most_recent_dt['date']).dt.days, np.nan)
 
    # Iterate through each row and get the key-value pair with the maximum value in the sizes dictionary
    for index, row in most_recent_dt.iterrows():
        
        sizes = row['sizes']
        if sizes:
            max_size = max(sizes, key=sizes.get)
            

            # Create additional columns for each previous column by history type
            for i in range(1, 4):
                
                if row['history_type'] == 'queries':
                    most_recent_dt.at[index, f'{row["history_type"]}_mostRecentCommonSize'] = max_size
                    most_recent_dt.at[index, f'{row["history_type"]}_mostRecentCommonSize_CountOfEvidence'] = sizes[max_size]
                    most_recent_dt.at[index, 'filters_mostRecentCommonSize'] = np.nan
                    most_recent_dt.at[index, 'filters_mostRecentCommonSize_CountOfEvidence'] = np.nan
                    most_recent_dt.at[index, 'transactions_mostRecentCommonSize'] = np.nan
                    most_recent_dt.at[index, 'transactions_mostRecentCommonSize_CountOfEvidence'] = np.nan
                elif row['history_type'] == 'filters':
                    most_recent_dt.at[index, f'{row["history_type"]}_mostRecentCommonSize'] = max_size
                    most_recent_dt.at[index, f'{row["history_type"]}_mostRecentCommonSize_CountOfEvidence'] = sizes[max_size]
                    most_recent_dt.at[index, 'queries_mostRecentCommonSize'] = np.nan
                    most_recent_dt.at[index, 'queries_mostRecentCommonSize_CountOfEvidence'] = np.nan
                    most_recent_dt.at[index, 'transactions_mostRecentCommonSize'] = np.nan
                    most_recent_dt.at[index, 'transactions_mostRecentCommonSize_CountOfEvidence'] = np.nan
                elif row['history_type'] == 'transactions':
                    most_recent_dt.at[index, f'{row["history_type"]}_mostRecentCommonSize'] = max_size
                    most_recent_dt.at[index, f'{row["history_type"]}_mostRecentCommonSize_CountOfEvidence'] = sizes[max_size]
                    most_recent_dt.at[index, 'queries_mostRecentCommonSize'] = np.nan
                    most_recent_dt.at[index, 'queries_mostRecentCommonSize_CountOfEvidence'] = np.nan
                    most_recent_dt.at[index, 'filters_mostRecentCommonSize'] = np.nan
                    most_recent_dt.at[index, 'filters_mostRecentCommonSize_CountOfEvidence'] = np.nan
    # Convert 'sizes' column from Counter to tuple
    most_recent_dt['sizes'] = most_recent_dt['sizes'].apply(lambda x: tuple(x.items()))

  
    # Group by the desired columns and keep the values after the max
    most_recent_dt = most_recent_dt.groupby(['user_id', 'leaf_id', 'label_trx_dt', 'label_trx_size']).agg({
        'queries_mostRecentCommonSize_DaysDiff': 'last',
        'filters_mostRecentCommonSize_DaysDiff': 'last',
        'transactions_mostRecentCommonSize_DaysDiff': 'last',
        
        'queries_mostRecentCommonSize': 'last',
        'filters_mostRecentCommonSize': 'last',
        'transactions_mostRecentCommonSize': 'last',
        
        'queries_mostRecentCommonSize_CountOfEvidence': 'last',
        'filters_mostRecentCommonSize_CountOfEvidence': 'last',
        'transactions_mostRecentCommonSize_CountOfEvidence': 'last'})  
    
    
    most_recent_dt.reset_index( inplace=True) 
    # Fill None values with NaN
    most_recent_dt.fillna(np.nan, inplace=True)    
    return most_recent_dt

In [155]:
sources_most_recent_size_diff=history_signals_date_mostRecentSize_diff( queries, filters, transactions,
                                                                       df1_history_type='queries', df2_history_type='filters', df3_history_type='transactions')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [156]:
sources_most_recent_size_diff.head(10)

,user_id,leaf_id,label_trx_dt,label_trx_size,queries_mostRecentCommonSize_DaysDiff,filters_mostRecentCommonSize_DaysDiff,transactions_mostRecentCommonSize_DaysDiff,queries_mostRecentCommonSize,filters_mostRecentCommonSize,transactions_mostRecentCommonSize,queries_mostRecentCommonSize_CountOfEvidence,filters_mostRecentCommonSize_CountOfEvidence,transactions_mostRecentCommonSize_CountOfEvidence
0,60,15709,2023-04-05,13.0,NaN,NaN,46.0,NaN,NaN,5.0,NaN,NaN,1.0
1,60,95672,2023-03-23,6.5,14.0,22.0,8.0,6.5,6.5,6.5,5.0,5.0,1.0
2,77,95672,2022-10-23,8.5,NaN,NaN,7.0,NaN,NaN,8.5,NaN,NaN,1.0
3,192,95672,2022-06-24,8.5,NaN,NaN,34.0,NaN,NaN,8.5,NaN,NaN,1.0
4,2520,15709,2023-01-07,13.0,NaN,NaN,99.0,NaN,NaN,12.0,NaN,NaN,1.0
5,2642,15709,2022-10-25,10.5,NaN,NaN,2.0,NaN,NaN,10.5,NaN,NaN,2.0
6,2741,95672,2023-02-21,8.5,NaN,NaN,219.0,NaN,NaN,8.5,NaN,NaN,2.0
7,3285,95672,2023-02-19,8.0,NaN,NaN,108.0,NaN,NaN,8.0,NaN,NaN,1.0
8,3751,15709,2022-12-31,10.0,NaN,NaN,1.0,NaN,NaN,10.0,NaN,NaN,1.0
9,4163,15709,2023-02-07,9.5,NaN,NaN,49.0,NaN,NaN,9.5,NaN,NaN,1.0


In [157]:
sources_most_recent_size_diff[(sources_most_recent_size_diff['queries_mostRecentCommonSize']!= sources_most_recent_size_diff['filters_mostRecentCommonSize'])& 
                              (sources_most_recent_size_diff['transactions_mostRecentCommonSize']!= sources_most_recent_size_diff['filters_mostRecentCommonSize'])& 
                              (sources_most_recent_size_diff.filters_mostRecentCommonSize.notna())& 
                              (sources_most_recent_size_diff.transactions_mostRecentCommonSize.notna()) &
                               (sources_most_recent_size_diff.queries_mostRecentCommonSize.notna())].head(2)

,user_id,leaf_id,label_trx_dt,label_trx_size,queries_mostRecentCommonSize_DaysDiff,filters_mostRecentCommonSize_DaysDiff,transactions_mostRecentCommonSize_DaysDiff,queries_mostRecentCommonSize,filters_mostRecentCommonSize,transactions_mostRecentCommonSize,queries_mostRecentCommonSize_CountOfEvidence,filters_mostRecentCommonSize_CountOfEvidence,transactions_mostRecentCommonSize_CountOfEvidence
335,117278,95672,2023-03-31,12.0,6.0,4.0,335.0,12.0,11.5,12.0,1.0,1.0,1.0
450,164133,15709,2023-03-16,10.5,54.0,19.0,18.0,10.0,10.5,10.0,1.0,1.0,1.0


In [158]:
sources_most_recent_size_diff[(sources_most_recent_size_diff['user_id']== 5242)].head(2)

,user_id,leaf_id,label_trx_dt,label_trx_size,queries_mostRecentCommonSize_DaysDiff,filters_mostRecentCommonSize_DaysDiff,transactions_mostRecentCommonSize_DaysDiff,queries_mostRecentCommonSize,filters_mostRecentCommonSize,transactions_mostRecentCommonSize,queries_mostRecentCommonSize_CountOfEvidence,filters_mostRecentCommonSize_CountOfEvidence,transactions_mostRecentCommonSize_CountOfEvidence
13,5242,15709,2023-02-16,10.0,NaN,6.0,NaN,NaN,10.0,NaN,NaN,1.0,NaN


In [159]:
queries[(queries['user_id']== 5242)].head(2)

,user_id,leaf_id,date,sizes,label_trx_dt,label_trx_size,history_type


In [160]:
most_recent_size_diff[(most_recent_size_diff['user_id']== 5242)].head(2)

,user_id,leaf_id,sizes,date,label_trx_dt,label_trx_size,Rank_dt,mostRecentCommonSize_DaysDiff,mostRecentCommonSize,mostRecentCommonSize_CountOfEvidence
809763,5242,15709,{10.0: 1},2023-02-10,2023-02-16,10.0,1,6,10.0,1.0


In [161]:
sources_most_recent_size_diff.to_pickle('/data/ebay/data/olivyatan/size_data/sources_most_recent_size_diff_030723.pkl')
most_recent_size_diff.to_pickle('/data/ebay/data/olivyatan/size_data/most_recent_size_diff_030723.pkl')

###### user id== 60 examination: 

In [290]:
sources_most_recent_size_diff=history_signals_date_mostRecentSize_diff(transactions[transactions['user_id']==60], filters[filters['user_id']==60], queries[queries['user_id']==60])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [291]:
sources_most_recent_size_diff

,user_id,leaf_id,label_trx_dt,label_trx_size,queries_MostRecentCommonSize_DaysDiff,filters_MostRecentCommonSize_DaysDiff,transactions_MostRecentCommonSize_DaysDiff,queries_mostRecentCommonSize,filters_mostRecentCommonSize,transactions_mostRecentCommonSize,queries_mostRecentCommonSize_CountOfEvidence,filters_mostRecentCommonSize_CountOfEvidence,transactions_mostRecentCommonSize_CountOfEvidence
0,60,15709,2023-04-05,13.0,46.0,46.0,NaN,5.0,5.0,NaN,1.0,1.0,NaN
1,60,95672,2023-03-23,6.5,8.0,8.0,14.0,6.5,6.5,6.5,1.0,1.0,5.0


In [292]:
most_recent_dt

,user_id,leaf_id,date,sizes,label_trx_dt,label_trx_size,Rank_dt,MostRecentSize_DaysDiff,mostRecentSize1,mostRecentSize_CountOfEvidence1
5,60,15709,2023-02-18,{5.0: 1},2023-04-05,13.0,1,46,5.0,1
13,60,95672,2023-03-15,{6.5: 1},2023-03-23,6.5,1,8,6.5,1


### Combine features of sources stats, aggregated time features and aggregate history type time features : 

In [162]:
merged_label_srcs.head(4)

,user_id,leaf_id,merged_sizes_all_srcs,merged_std,merged_mean,merged_median,merged_mode,merged_maxMinDiff,merged_countOfEvidence,transactions_std,...,queries_countOfEvidence,filters_std,filters_mean,filters_median,filters_mode,filters_maxMinDiff,filters_countOfEvidence,label_trx_dt,label_user_size,label_trx_size
0,60,15709,"{12.0: 1, 12.5: 1, 7.0: 1, 13.0: 1, 5.0: 1}",3.261901,9.900000,12.0,5.0,8.0,5,3.261901,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-05,{13.0: 1},13.0
1,60,95672,"{6.5: 28, 8.0: 1, 8.5: 1}",0.441273,6.616667,6.5,6.5,2.0,30,0.801784,...,5.0,0.0,6.5,6.5,6.5,0.0,18.0,2023-03-23,{6.5: 1},6.5
2,77,95672,{8.5: 2},0.000000,8.500000,8.5,8.5,0.0,2,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-23,{8.5: 1},8.5
3,192,95672,{8.5: 1},0.000000,8.500000,8.5,8.5,0.0,1,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-24,{8.5: 1},8.5


In [163]:
merged_label_srcs.columns

Index(['user_id', 'leaf_id', 'merged_sizes_all_srcs', 'merged_std',
       'merged_mean', 'merged_median', 'merged_mode', 'merged_maxMinDiff',
       'merged_countOfEvidence', 'transactions_std', 'transactions_mean',
       'transactions_median', 'transactions_mode', 'transactions_maxMinDiff',
       'transactions_countOfEvidence', 'queries_std', 'queries_mean',
       'queries_median', 'queries_mode', 'queries_maxMinDiff',
       'queries_countOfEvidence', 'filters_std', 'filters_mean',
       'filters_median', 'filters_mode', 'filters_maxMinDiff',
       'filters_countOfEvidence', 'label_trx_dt', 'label_user_size',
       'label_trx_size'],
      dtype='object')

In [164]:
most_recent_size_diff.head(3) ## date is the most recent date of all sources.  

,user_id,leaf_id,sizes,date,label_trx_dt,label_trx_size,Rank_dt,mostRecentCommonSize_DaysDiff,mostRecentCommonSize,mostRecentCommonSize_CountOfEvidence
0,14355,15709,{10.5: 1},2023-02-17,2023-03-25,10.5,1,36,10.5,1.0
2,18346,15709,{11.0: 1},2023-02-09,2023-02-10,11.0,1,1,11.0,1.0
3,19016,15709,{10.0: 1},2023-03-05,2023-03-24,10.0,1,19,10.0,1.0


In [165]:
sources_most_recent_size_diff[sources_most_recent_size_diff['user_id']==60].head(4)

,user_id,leaf_id,label_trx_dt,label_trx_size,queries_mostRecentCommonSize_DaysDiff,filters_mostRecentCommonSize_DaysDiff,transactions_mostRecentCommonSize_DaysDiff,queries_mostRecentCommonSize,filters_mostRecentCommonSize,transactions_mostRecentCommonSize,queries_mostRecentCommonSize_CountOfEvidence,filters_mostRecentCommonSize_CountOfEvidence,transactions_mostRecentCommonSize_CountOfEvidence
0,60,15709,2023-04-05,13.0,NaN,NaN,46.0,NaN,NaN,5.0,NaN,NaN,1.0
1,60,95672,2023-03-23,6.5,14.0,22.0,8.0,6.5,6.5,6.5,5.0,5.0,1.0


In [166]:
sources_most_recent_size_diff[sources_most_recent_size_diff['user_id']==60]

,user_id,leaf_id,label_trx_dt,label_trx_size,queries_mostRecentCommonSize_DaysDiff,filters_mostRecentCommonSize_DaysDiff,transactions_mostRecentCommonSize_DaysDiff,queries_mostRecentCommonSize,filters_mostRecentCommonSize,transactions_mostRecentCommonSize,queries_mostRecentCommonSize_CountOfEvidence,filters_mostRecentCommonSize_CountOfEvidence,transactions_mostRecentCommonSize_CountOfEvidence
0,60,15709,2023-04-05,13.0,NaN,NaN,46.0,NaN,NaN,5.0,NaN,NaN,1.0
1,60,95672,2023-03-23,6.5,14.0,22.0,8.0,6.5,6.5,6.5,5.0,5.0,1.0


In [167]:
print(merged_label_srcs[merged_label_srcs['user_id']==60])

  user_id leaf_id                        merged_sizes_all_srcs  merged_std  \
0      60   15709  {12.0: 1, 12.5: 1, 7.0: 1, 13.0: 1, 5.0: 1}    3.261901   
1      60   95672                    {6.5: 28, 8.0: 1, 8.5: 1}    0.441273   

   merged_mean  merged_median  merged_mode  merged_maxMinDiff  \
0     9.900000           12.0          5.0                8.0   
1     6.616667            6.5          6.5                2.0   

   merged_countOfEvidence  transactions_std  ...  queries_countOfEvidence  \
0                       5          3.261901  ...                      NaN   
1                      30          0.801784  ...                      5.0   

   filters_std  filters_mean  filters_median  filters_mode  \
0          NaN           NaN             NaN           NaN   
1          0.0           6.5             6.5           6.5   

   filters_maxMinDiff  filters_countOfEvidence  label_trx_dt  label_user_size  \
0                 NaN                      NaN    2023-04-05        {

In [168]:
transactions[transactions['user_id']==60]

,user_id,leaf_id,date,sizes,label_trx_dt,label_trx_size,history_type
1,60,15709,2022-05-15,{12.0: 1},2023-04-05,13.0,transactions
2,60,15709,2022-09-21,{12.5: 1},2023-04-05,13.0,transactions
3,60,15709,2022-11-20,{7.0: 1},2023-04-05,13.0,transactions
4,60,15709,2023-01-04,{13.0: 1},2023-04-05,13.0,transactions
5,60,15709,2023-02-18,{5.0: 1},2023-04-05,13.0,transactions
7,60,95672,2022-05-14,{8.0: 1},2023-03-23,6.5,transactions
8,60,95672,2022-12-05,{6.5: 1},2023-03-23,6.5,transactions
9,60,95672,2022-12-22,{6.5: 1},2023-03-23,6.5,transactions
10,60,95672,2023-01-08,{6.5: 1},2023-03-23,6.5,transactions
11,60,95672,2023-02-03,{6.5: 1},2023-03-23,6.5,transactions


In [169]:
queries[queries['user_id']==60]

,user_id,leaf_id,date,sizes,label_trx_dt,label_trx_size,history_type
3328645,60,95672,2023-03-09,{6.5: 5},2023-03-23,6.5,queries


In [170]:
filters[filters['user_id']==60]

,user_id,leaf_id,date,sizes,label_trx_dt,label_trx_size,history_type
5791546,60,95672,2023-02-15,{6.5: 6},2023-03-23,6.5,filters
5791547,60,95672,2023-02-16,{6.5: 1},2023-03-23,6.5,filters
5791548,60,95672,2023-02-23,{6.5: 1},2023-03-23,6.5,filters
5791549,60,95672,2023-02-24,{6.5: 2},2023-03-23,6.5,filters
5791550,60,95672,2023-02-26,{6.5: 2},2023-03-23,6.5,filters
5791551,60,95672,2023-02-27,{6.5: 1},2023-03-23,6.5,filters
5791552,60,95672,2023-03-01,{6.5: 5},2023-03-23,6.5,filters


In [171]:
df_merge_queries_per_user_categ[df_merge_queries_per_user_categ['user_id']==77]

,user_id,leaf_id,sizes,queries_std,queries_mean,queries_median,queries_mode,queries_maxMinDiff,queries_countOfEvidence


In [172]:
df_merge_filters_per_user_categ[df_merge_filters_per_user_categ['user_id']==77]

,user_id,leaf_id,sizes,filters_std,filters_mean,filters_median,filters_mode,filters_maxMinDiff,filters_countOfEvidence


In [173]:
df_merge_transactions_per_user_categ[df_merge_transactions_per_user_categ['user_id']==77]

,user_id,leaf_id,sizes,transactions_std,transactions_mean,transactions_median,transactions_mode,transactions_maxMinDiff,transactions_countOfEvidence
2,77,95672,{8.5: 2},0.0,8.5,8.5,8.5,0.0,2


In [174]:
merged_label_srcs_features = pd.merge(merged_label_srcs, most_recent_size_diff,  on=['user_id','leaf_id'], how='left')
merged_label_srcs_features = pd.merge(merged_label_srcs_features, sources_most_recent_size_diff, on=['user_id','leaf_id'], how='left')

In [175]:
merged_label_srcs_features.head(5) 

,user_id,leaf_id,merged_sizes_all_srcs,merged_std,merged_mean,merged_median,merged_mode,merged_maxMinDiff,merged_countOfEvidence,transactions_std,...,label_trx_size,queries_mostRecentCommonSize_DaysDiff,filters_mostRecentCommonSize_DaysDiff,transactions_mostRecentCommonSize_DaysDiff,queries_mostRecentCommonSize,filters_mostRecentCommonSize,transactions_mostRecentCommonSize,queries_mostRecentCommonSize_CountOfEvidence,filters_mostRecentCommonSize_CountOfEvidence,transactions_mostRecentCommonSize_CountOfEvidence
0,60,15709,"{12.0: 1, 12.5: 1, 7.0: 1, 13.0: 1, 5.0: 1}",3.261901,9.900000,12.0,5.0,8.0,5,3.261901,...,13.0,NaN,NaN,46.0,NaN,NaN,5.0,NaN,NaN,1.0
1,60,95672,"{6.5: 28, 8.0: 1, 8.5: 1}",0.441273,6.616667,6.5,6.5,2.0,30,0.801784,...,6.5,14.0,22.0,8.0,6.5,6.5,6.5,5.0,5.0,1.0
2,77,95672,{8.5: 2},0.000000,8.500000,8.5,8.5,0.0,2,0.000000,...,8.5,NaN,NaN,7.0,NaN,NaN,8.5,NaN,NaN,1.0
3,192,95672,{8.5: 1},0.000000,8.500000,8.5,8.5,0.0,1,0.000000,...,8.5,NaN,NaN,34.0,NaN,NaN,8.5,NaN,NaN,1.0
4,2520,15709,{12.0: 1},0.000000,12.000000,12.0,12.0,0.0,1,0.000000,...,13.0,NaN,NaN,99.0,NaN,NaN,12.0,NaN,NaN,1.0


In [176]:
merged_label_srcs_features.columns

Index(['user_id', 'leaf_id', 'merged_sizes_all_srcs', 'merged_std',
       'merged_mean', 'merged_median', 'merged_mode', 'merged_maxMinDiff',
       'merged_countOfEvidence', 'transactions_std', 'transactions_mean',
       'transactions_median', 'transactions_mode', 'transactions_maxMinDiff',
       'transactions_countOfEvidence', 'queries_std', 'queries_mean',
       'queries_median', 'queries_mode', 'queries_maxMinDiff',
       'queries_countOfEvidence', 'filters_std', 'filters_mean',
       'filters_median', 'filters_mode', 'filters_maxMinDiff',
       'filters_countOfEvidence', 'label_trx_dt_x', 'label_user_size',
       'label_trx_size_x', 'sizes', 'date', 'label_trx_dt_y',
       'label_trx_size_y', 'Rank_dt', 'mostRecentCommonSize_DaysDiff',
       'mostRecentCommonSize', 'mostRecentCommonSize_CountOfEvidence',
       'label_trx_dt', 'label_trx_size',
       'queries_mostRecentCommonSize_DaysDiff',
       'filters_mostRecentCommonSize_DaysDiff',
       'transactions_mostRecentC

In [177]:
column_order = ['user_id', 'leaf_id', 'merged_sizes_all_srcs', 'merged_std',
       'merged_mean', 'merged_median', 'merged_mode', 'merged_maxMinDiff',
       'merged_countOfEvidence', 'transactions_std', 'transactions_mean',
       'transactions_median', 'transactions_mode', 'transactions_maxMinDiff',
       'transactions_countOfEvidence', 'queries_std', 'queries_mean',
       'queries_median', 'queries_mode', 'queries_maxMinDiff',
       'queries_countOfEvidence', 'filters_std', 'filters_mean',
       'filters_median', 'filters_mode', 'filters_maxMinDiff',
       'filters_countOfEvidence',
       'mostRecentCommonSize_DaysDiff','mostRecentCommonSize', 'mostRecentCommonSize_CountOfEvidence',        
       'queries_mostRecentCommonSize_DaysDiff',
       'filters_mostRecentCommonSize_DaysDiff',
       'transactions_mostRecentCommonSize_DaysDiff',
       'queries_mostRecentCommonSize', 
       'filters_mostRecentCommonSize',
       'transactions_mostRecentCommonSize',
       'queries_mostRecentCommonSize_CountOfEvidence',
       'filters_mostRecentCommonSize_CountOfEvidence',
       'transactions_mostRecentCommonSize_CountOfEvidence',
       'label_trx_dt','label_user_size', 'label_trx_size']



In [178]:
merged_label_srcs_features = merged_label_srcs_features[column_order]

In [179]:
merged_label_srcs_features.head(20)

,user_id,leaf_id,merged_sizes_all_srcs,merged_std,merged_mean,merged_median,merged_mode,merged_maxMinDiff,merged_countOfEvidence,transactions_std,...,transactions_mostRecentCommonSize_DaysDiff,queries_mostRecentCommonSize,filters_mostRecentCommonSize,transactions_mostRecentCommonSize,queries_mostRecentCommonSize_CountOfEvidence,filters_mostRecentCommonSize_CountOfEvidence,transactions_mostRecentCommonSize_CountOfEvidence,label_trx_dt,label_user_size,label_trx_size
0,60,15709,"{12.0: 1, 12.5: 1, 7.0: 1, 13.0: 1, 5.0: 1}",3.261901,9.900000,12.0,5.0,8.0,5,3.261901,...,46.0,NaN,NaN,5.0,NaN,NaN,1.0,2023-04-05,{13.0: 1},13.0
1,60,95672,"{6.5: 28, 8.0: 1, 8.5: 1}",0.441273,6.616667,6.5,6.5,2.0,30,0.801784,...,8.0,6.5,6.5,6.5,5.0,5.0,1.0,2023-03-23,{6.5: 1},6.5
2,77,95672,{8.5: 2},0.000000,8.500000,8.5,8.5,0.0,2,0.000000,...,7.0,NaN,NaN,8.5,NaN,NaN,1.0,2022-10-23,{8.5: 1},8.5
3,192,95672,{8.5: 1},0.000000,8.500000,8.5,8.5,0.0,1,0.000000,...,34.0,NaN,NaN,8.5,NaN,NaN,1.0,2022-06-24,{8.5: 1},8.5
4,2520,15709,{12.0: 1},0.000000,12.000000,12.0,12.0,0.0,1,0.000000,...,99.0,NaN,NaN,12.0,NaN,NaN,1.0,2023-01-07,{13.0: 1},13.0
5,2642,15709,"{11.0: 1, 10.5: 3}",0.216506,10.625000,10.5,10.5,0.5,4,0.216506,...,2.0,NaN,NaN,10.5,NaN,NaN,2.0,2022-10-25,{10.5: 1},10.5
6,2741,95672,{8.5: 2},0.000000,8.500000,8.5,8.5,0.0,2,0.000000,...,219.0,NaN,NaN,8.5,NaN,NaN,2.0,2023-02-21,{8.5: 1},8.5
7,3285,95672,{8.0: 1},0.000000,8.000000,8.0,8.0,0.0,1,0.000000,...,108.0,NaN,NaN,8.0,NaN,NaN,1.0,2023-02-19,{8.0: 1},8.0
8,3751,15709,{10.0: 5},0.000000,10.000000,10.0,10.0,0.0,5,0.000000,...,1.0,NaN,NaN,10.0,NaN,NaN,1.0,2022-12-31,{10.0: 3},10.0
9,4163,15709,{9.5: 2},0.000000,9.500000,9.5,9.5,0.0,2,0.000000,...,49.0,NaN,NaN,9.5,NaN,NaN,1.0,2023-02-07,{9.5: 1},9.5


In [180]:
filters[filters['user_id']==77]

,user_id,leaf_id,date,sizes,label_trx_dt,label_trx_size,history_type


In [181]:
sources_most_recent_size_diff[(sources_most_recent_size_diff['user_id']== 77)].head(2)

,user_id,leaf_id,label_trx_dt,label_trx_size,queries_mostRecentCommonSize_DaysDiff,filters_mostRecentCommonSize_DaysDiff,transactions_mostRecentCommonSize_DaysDiff,queries_mostRecentCommonSize,filters_mostRecentCommonSize,transactions_mostRecentCommonSize,queries_mostRecentCommonSize_CountOfEvidence,filters_mostRecentCommonSize_CountOfEvidence,transactions_mostRecentCommonSize_CountOfEvidence
2,77,95672,2022-10-23,8.5,NaN,NaN,7.0,NaN,NaN,8.5,NaN,NaN,1.0


In [182]:
sources_most_recent_size_diff[(sources_most_recent_size_diff['user_id']== 3751)].head(2)

,user_id,leaf_id,label_trx_dt,label_trx_size,queries_mostRecentCommonSize_DaysDiff,filters_mostRecentCommonSize_DaysDiff,transactions_mostRecentCommonSize_DaysDiff,queries_mostRecentCommonSize,filters_mostRecentCommonSize,transactions_mostRecentCommonSize,queries_mostRecentCommonSize_CountOfEvidence,filters_mostRecentCommonSize_CountOfEvidence,transactions_mostRecentCommonSize_CountOfEvidence
8,3751,15709,2022-12-31,10.0,NaN,NaN,1.0,NaN,NaN,10.0,NaN,NaN,1.0


In [183]:
merged_label_srcs_features[merged_label_srcs_features['user_id']==77][['filters_mean', 'filters_mode','filters_countOfEvidence','filters_mostRecentCommonSize',
                                                                       'queries_mostRecentCommonSize',
                                                                       'filters_mostRecentCommonSize_DaysDiff','queries_countOfEvidence', 'transactions_countOfEvidence']]

,filters_mean,filters_mode,filters_countOfEvidence,filters_mostRecentCommonSize,queries_mostRecentCommonSize,filters_mostRecentCommonSize_DaysDiff,queries_countOfEvidence,transactions_countOfEvidence
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0


In [184]:
sources_most_recent_size_diff[
                              (merged_label_srcs_features.filters_mean.isna())& 
                              (merged_label_srcs_features.filters_mostRecentCommonSize.notna())] 

,user_id,leaf_id,label_trx_dt,label_trx_size,queries_mostRecentCommonSize_DaysDiff,filters_mostRecentCommonSize_DaysDiff,transactions_mostRecentCommonSize_DaysDiff,queries_mostRecentCommonSize,filters_mostRecentCommonSize,transactions_mostRecentCommonSize,queries_mostRecentCommonSize_CountOfEvidence,filters_mostRecentCommonSize_CountOfEvidence,transactions_mostRecentCommonSize_CountOfEvidence


In [ ]:
#bug

In [185]:
filters[filters['user_id']==77]

,user_id,leaf_id,date,sizes,label_trx_dt,label_trx_size,history_type


In [186]:
df_merge_filters_per_user_categ[df_merge_filters_per_user_categ['user_id']==77]

,user_id,leaf_id,sizes,filters_std,filters_mean,filters_median,filters_mode,filters_maxMinDiff,filters_countOfEvidence


In [187]:
merged_label_srcs[merged_label_srcs['user_id']==77]

,user_id,leaf_id,merged_sizes_all_srcs,merged_std,merged_mean,merged_median,merged_mode,merged_maxMinDiff,merged_countOfEvidence,transactions_std,...,queries_countOfEvidence,filters_std,filters_mean,filters_median,filters_mode,filters_maxMinDiff,filters_countOfEvidence,label_trx_dt,label_user_size,label_trx_size
2,77,95672,{8.5: 2},0.0,8.5,8.5,8.5,0.0,2,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-23,{8.5: 1},8.5


In [188]:
merged_label_srcs_features[(merged_label_srcs_features.filters_mean.notna())& (merged_label_srcs_features.filters_mostRecentCommonSize.isna())][['user_id', 'leaf_id','filters_mean',\
                                                                                       'filters_mean', 'filters_mode','filters_countOfEvidence','filters_mostRecentCommonSize',\
                                                                                      'filters_mostRecentCommonSize_DaysDiff','queries_countOfEvidence', 'transactions_countOfEvidence' ]]

,user_id,leaf_id,filters_mean,filters_mean,filters_mode,filters_countOfEvidence,filters_mostRecentCommonSize,filters_mostRecentCommonSize_DaysDiff,queries_countOfEvidence,transactions_countOfEvidence


In [191]:
# Calculate the percentage of non-null values for each column
percentage_not_na = merged_label_srcs_features.count() / len(merged_label_srcs_features) * 100

# Print the results
print(percentage_not_na)

user_id                                              100.000000
leaf_id                                              100.000000
merged_sizes_all_srcs                                100.000000
merged_std                                           100.000000
merged_mean                                          100.000000
merged_median                                        100.000000
merged_mode                                          100.000000
merged_maxMinDiff                                    100.000000
merged_countOfEvidence                               100.000000
transactions_std                                      87.868362
transactions_mean                                     87.868362
transactions_median                                   87.868362
transactions_mode                                     87.868362
transactions_maxMinDiff                               87.868362
transactions_countOfEvidence                          87.868362
queries_std                             

In [193]:
merged_label_srcs_features.to_pickle('/data/ebay/data/olivyatan/size_data/merged_label_srcs_time_features_030723.pkl')

In [ ]:
merged_label_srcs_features.column

#### prev logic

In [ ]:
df_merge_transactions_per_user_categ=df_merge_transactions_per_user_categ.copy() 
df_merge_queries_per_user_categ_str=df_merge_queries_per_user_categ.copy() 
df_merge_filters_per_user_categ_str=df_merge_filters_per_user_categ.copy()  

df_agg_trx_per_user_categ_str.user_sizes=[str(dict(i)) for i in df_agg_trx_per_user.user_sizes]
df_agg_queries_per_categ_user_str.user_sizes=[str(dict(i)) for i in df_agg_queries_per_user.user_sizes] 
df_agg_filters_per_categ_user_str.user_sizes=[str(dict(i)) for i in df_agg_filters_per_user.user_sizes]

df_agg_trx_per_user_categ_str.to_parquet('/data/ebay/data/olivyatan/size_data/trx_agg_per_user_3m_19.4.23.parquet')
df_agg_queries_per_categ_user_str.to_parquet('/data/ebay/data/olivyatan/size_data/queries_agg_per_user_3m_19.4.23.parquet')
df_agg_filters_per_categ_user_str.to_parquet('/data/ebay/data/olivyatan/size_data/filters_agg_per_user_3m_19.4.23.parquet')



In [744]:
df_agg_trx_per_user_str=df_agg_trx_per_user.copy() 
df_agg_trx_per_user_str.sizes=[str(dict(i)) for i in df_agg_trx_per_user.sizes]
df_agg_trx_per_user_str.to_parquet('/data/ebay/data/olivyatan/size_data/trx_agg_per_user_3m_19.4.23.parquet')

In [ ]:
df_agg_trx_per_user.shape

(2548876, 11)

In [818]:
df_agg_filters_per_user[df_agg_filters_per_user['user_id']==77].head(1)

,dominant_category_id,user_id,session_start_dt,user_sizes,filters_std,filters_mean,filters_median,filters_mode,filters_maxMinDiff,filters_countOfEvidence


In [820]:
df_merge_queries_per_user_categ[df_merge_queries_per_user_categ['user_id']==77].head(1)

,user_id,leaf_id,sizes,queries_std,queries_mean,queries_median,queries_mode,queries_maxMinDiff,queries_countOfEvidence


In [821]:
df_merge_transactions_per_user_categ[df_merge_transactions_per_user_categ['user_id']==77].head(1)

,user_id,leaf_id,sizes,transactions_std,transactions_mean,transactions_median,transactions_mode,transactions_maxMinDiff,transactions_countOfEvidence
1,77,95672,{8.5: 2},0.0,8.5,8.5,8.5,0.0,2


In [ ]:
#combine as features: 

In [ ]:
def merge_history_src_signals(df1: pd.DataFrame,sizes_col: str ):
   
    df_merged = df1.groupby(['user_id','leaf_id']).apply(lambda x: merge_counter_sizes(x['sizes_col'])).reset_index()
    df_merged.columns = ['user_id', 'leaf_id','sizes_col']

    print(f"DataFrame shape following merging sizes counts: {df_merged.shape}")

    df_agg = agg_sizes(df_merged, 'sizes', col_prefix='merged')

    return df_agg

In [ ]:
#men_merged_signals = 
m=merge_signals(queries[queries['user_id']==60] , filters[filters['user_id']==60], transactions[transactions['user_id']==60])

In [ ]:
def size_per_site_and_categ(df: pd.DataFrame, site_id: int, leaf_id: int, max_size: float, min_size: float,
                            history_type: str = 'filter', site_id_col: str = 'site_id', leaf_id_col='leaf_category_id',
                            user_id_col='user_id', secondary_regex=None) -> Optional[pd.DataFrame]:

In [ ]:
women_trx = size_per_site_and_categ(df=trx_df,
                                            site_id=site_id,
                                            leaf_id=women_sneakers_leaf_id,
                                            max_size=self.women_max_size,
                                            min_size=self.women_min_size,
                                            history_type='transactions',
                                            site_id_col='SITE_ID',
                                            leaf_id_col='LEAF_CATEG_ID',
                                            user_id_col='BUYER_ID')

In [462]:
site_id=0
leaf_id=15709
max_size = 16
min_size = 4.5
user_id_col = "user_id"
date_col = 'session_start_dt'
categ_col = 'dominant_category_id'
women_queries = size_per_site_and_categ(queries_df.head(300), site_id,leaf_id, max_size,min_size,\
                                        history_type='queries',site_id_col='site_id',leaf_id_col='dominant_category_id',user_id_col='user_id')

Output df shape for leaf_id: 15709, history type: queries - (249, 8)


In [ ]:
### from filters: create per user : 

In [511]:
df_per_user = pd.DataFrame({"user_id": filters_per_user_date_categ_groupby.user_id.astype(int), "user_sizes": filters_per_user_date_categ_groupby.user_sizes})

In [512]:
df_per_user

,user_id,user_sizes
0,192,{8.5: 1}
1,376,{9.5: 1}
2,526,{12.0: 2}
3,847,{10.0: 1}
4,862,{9.0: 1}
...,...,...
6777198,2516074270,"{11.5: 1, 12.0: 1}"
6777199,2516078377,{7.5: 1}
6777200,2516086497,{8.0: 2}
6777201,2516087333,{9.5: 1}


In [513]:
df_per_user = df_per_user[df_per_user.user_sizes.astype(bool)]

In [514]:
df_per_user.user_sizes.iloc[0]

Counter({8.5: 1})

In [515]:
df_per_user

,user_id,user_sizes
0,192,{8.5: 1}
1,376,{9.5: 1}
2,526,{12.0: 2}
3,847,{10.0: 1}
4,862,{9.0: 1}
...,...,...
6777196,2516071366,{10.0: 3}
6777198,2516074270,"{11.5: 1, 12.0: 1}"
6777199,2516078377,{7.5: 1}
6777200,2516086497,{8.0: 2}


In [516]:
df_agg = agg_sizes(df_per_user.head(100), 'user_sizes', col_prefix='filters')

In [517]:
df_agg

,user_id,user_sizes,filters_std,filters_mean,filters_median,filters_mode,filters_maxMinDiff,filters_countOfEvidence
0,192,{8.5: 1},0.00,8.50,8.50,8.5,0.0,1
1,376,{9.5: 1},0.00,9.50,9.50,9.5,0.0,1
2,526,{12.0: 2},0.00,12.00,12.00,12.0,0.0,2
3,847,{10.0: 1},0.00,10.00,10.00,10.0,0.0,1
4,862,{9.0: 1},0.00,9.00,9.00,9.0,0.0,1
...,...,...,...,...,...,...,...,...
97,8643,{11.0: 1},0.00,11.00,11.00,11.0,0.0,1
98,8643,{11.0: 1},0.00,11.00,11.00,11.0,0.0,1
99,8643,{11.0: 1},0.00,11.00,11.00,11.0,0.0,1
100,9004,{8.5: 5},0.00,8.50,8.50,8.5,0.0,5


In [503]:
trx_per_user_date_categ_groupby.head(100)

,BUYER_ID,LEAF_CATEG_ID,CREATED_DT,user_sizes
0,39,15709,2022-06-06,{9.5: 1}
1,60,15709,2022-05-15,{12.0: 1}
2,60,15709,2022-09-21,{12.5: 1}
3,60,15709,2022-11-20,{7.0: 1}
4,60,15709,2023-01-04,{13.0: 1}
...,...,...,...,...
95,5818,15709,2022-12-22,{10.0: 1}
96,5921,15709,2022-08-21,{11.5: 1}
97,5973,15709,2022-11-12,{13.0: 1}
98,5973,15709,2022-12-10,{13.0: 1}


In [500]:
trx_per_user_date_categ_groupby[trx_per_user_date_categ_groupby['BUYER_ID']==60].sort_values(by='CREATED_DT',ascending=False)

,BUYER_ID,LEAF_CATEG_ID,CREATED_DT,user_sizes
6,60,15709,2023-04-05,{13.0: 1}
14,60,95672,2023-03-23,{6.5: 1}
13,60,95672,2023-03-15,{6.5: 1}
5,60,15709,2023-02-18,{5.0: 1}
12,60,95672,2023-02-11,{8.5: 1}
11,60,95672,2023-02-03,{6.5: 1}
10,60,95672,2023-01-08,{6.5: 1}
4,60,15709,2023-01-04,{13.0: 1}
9,60,95672,2022-12-22,{6.5: 1}
8,60,95672,2022-12-05,{6.5: 1}


In [499]:
filters_per_user_date_categ_groupby[filters_per_user_date_categ_groupby['user_id']==60].sort_values(by='session_start_dt',ascending=False)

,dominant_category_id,user_id,session_start_dt,user_sizes
6102323,95672,60,2023-04-09,{6.5: 3}
6102322,95672,60,2023-04-08,{6.0: 1}
6102321,95672,60,2023-03-09,{}
6102320,95672,60,2023-03-01,{6.5: 5}
6102319,95672,60,2023-02-27,{6.5: 1}
6102318,95672,60,2023-02-26,{6.5: 2}
6102317,95672,60,2023-02-24,{6.5: 2}
6102316,95672,60,2023-02-23,{6.5: 1}
6102315,95672,60,2023-02-16,{6.5: 1}
6102314,95672,60,2023-02-15,{6.5: 6}


In [501]:
queries_per_user_date_categ_groupby[queries_per_user_date_categ_groupby['user_id']==60].sort_values(by='session_start_dt',ascending=False)

,dominant_category_id,user_id,session_start_dt,user_sizes
4709735,95672,60,2023-04-09,{}
4709734,95672,60,2023-04-08,{6.0: 1}
4709733,95672,60,2023-03-28,{}
4709732,95672,60,2023-03-25,{}
4709731,95672,60,2023-03-24,{}
4709730,95672,60,2023-03-22,{}
4709729,95672,60,2023-03-16,{}
4709728,95672,60,2023-03-14,{}
4709727,95672,60,2023-03-11,{}
4709726,95672,60,2023-03-09,{6.5: 5}


In [505]:
trx_per_user_date_categ_groupby[trx_per_user_date_categ_groupby['BUYER_ID']==5973].sort_values(by='CREATED_DT',ascending=False)

,BUYER_ID,LEAF_CATEG_ID,CREATED_DT,user_sizes
98,5973,15709,2022-12-10,{13.0: 1}
97,5973,15709,2022-11-12,{13.0: 1}


In [506]:
filters_per_user_date_categ_groupby[filters_per_user_date_categ_groupby['user_id']==5973].sort_values(by='session_start_dt',ascending=False)

,dominant_category_id,user_id,session_start_dt,user_sizes


In [507]:
queries_per_user_date_categ_groupby[queries_per_user_date_categ_groupby['user_id']==5973].sort_values(by='session_start_dt',ascending=False)

,dominant_category_id,user_id,session_start_dt,user_sizes


In [ ]:
#per user how many trx 1 year, how many queries 3m , how many filters 3m,min max date, avg 
#How many users has only 1 trx and no searches or filters? 


In [508]:

trx_per_user_date_categ_groupby[trx_per_user_date_categ_groupby['BUYER_ID']==5818].sort_values(by='CREATED_DT',ascending=False)

,BUYER_ID,LEAF_CATEG_ID,CREATED_DT,user_sizes
95,5818,15709,2022-12-22,{10.0: 1}


In [509]:
filters_per_user_date_categ_groupby[filters_per_user_date_categ_groupby['user_id']==5818].sort_values(by='session_start_dt',ascending=False)

,dominant_category_id,user_id,session_start_dt,user_sizes


In [510]:
queries_per_user_date_categ_groupby[queries_per_user_date_categ_groupby['user_id']==5818].sort_values(by='session_start_dt',ascending=False)

,dominant_category_id,user_id,session_start_dt,user_sizes


In [525]:
queries_per_user_date_categ_groupby.loc[0,'user_sizes']==Counter()


True

In [526]:
queries_per_user_date_categ_groupby.loc[1,'user_sizes']==Counter()

False

# Prev

In [23]:
queries_df = pd.read_parquet(f"hdfs://{tmp_output}/queries")

In [25]:
filters_df = pd.read_parquet(f"hdfs://{tmp_output}/filters")

In [26]:
queries_df.shape, filters_df.shape

((11303795, 7), (18043549, 7))

In [27]:
trx_df = pd.read_parquet(f"hdfs://{tmp_output}/trx")
trx_df.shape

(7600754, 10)

In [14]:
filters_df.session_start_dt.max(), filters_df.session_start_dt.min()

(datetime.date(2023, 4, 13), datetime.date(2023, 1, 15))

In [34]:
queries_df.session_start_dt.max(), queries_df.session_start_dt.min()

(datetime.date(2023, 4, 13), datetime.date(2023, 1, 15))

In [38]:
trx_df.CREATED_DT.max(), trx_df.CREATED_DT.min()

(datetime.date(2023, 4, 14), datetime.date(2022, 4, 15))

In [28]:
trx_df.head()

,BUYER_ID,ITEM_ID,ITEM_VRTN_ID,AUCT_END_DT,TRANSACTION_ID,LEAF_CATEG_ID,SITE_ID,SELLER_ID,ITEM_PRICE,CREATED_DT
0,1121682622,224804885145,-999,2022-04-16,2672758580012,15709,0,1073116591,79.99,2022-04-16
1,1763828331,303695909421,-999,2022-04-23,1955173626020,15709,0,1203869070,21.00,2022-04-23
2,1799019379,125301187300,-999,2022-06-11,2537469407002,15709,0,1696570392,48.00,2022-06-10
3,1861721938,125373010990,-999,2022-06-18,2539969033002,15709,0,1640169564,90.00,2022-06-18
4,1457404472,403781990707,-999,2022-07-24,0,15709,0,2383686637,140.00,2022-07-24


In [39]:
queries_df.head()

,site_id,session_skey,user_id,session_start_dt,clean_aspects,clean_query,dominant_category_id
0,0,38840641053920,2072725289,2023-01-30,,1985 jordan size 13,15709
1,0,38862920543100,2072725289,2023-02-25,,1985 jordan size 13,15709
2,0,38845168410750,1459699660,2023-02-04,,1985 size 9,15709
3,0,38897003577820,1802255695,2023-04-05,,2015 jordan 7 bordeaux size 10.5 preowned,15709
4,0,38871770347070,2029192040,2023-03-07,,919712 102 size 13,15709


In [29]:
filters_df.head()

,site_id,session_skey,user_id,session_start_dt,clean_aspects,clean_query,dominant_category_id
0,0,38873794853120,52214808,2023-03-09,brand:nike;performance/activity:cross training...,nike,15709
1,0,38884801053920,2340232373,2023-03-22,us shoe size:4,4 two 18 x air jordan 2 retro low,15709
2,0,38884717501800,1845226787,2023-03-22,us shoe size:14,jordan,15709
3,0,38890035693360,2426475243,2023-03-28,us shoe size:11,dunk sb,15709
4,0,38873577206710,1174812824,2023-03-09,color:multicolor;us shoe size:9,ultra boost,15709


# Process data 

In [54]:
filters_df.shape

(18043549, 7)

In [114]:
tmp_q = queries_df[queries_df.session_start_dt > datetime.date(2023, 4, 10)]
tmp_q.shape

(347600, 7)

In [56]:
tmp = filters_df[filters_df.session_start_dt > datetime.date(2023, 4, 10)]
tmp.shape

(559282, 7)

In [201]:
user_id_col = "user_id"
max_size = 16
min_size = 4.5
date_col = 'session_start_dt'
categ_col = 'dominant_category_id'

In [202]:
filters_per_user_date_categ_groupby = tmp.groupby([categ_col, user_id_col, date_col]).apply(
    lambda x: size_filters_per_user(x['clean_aspects'], max_size, min_size))

In [210]:
queries_per_user_date_categ_groupby = tmp_q.groupby([categ_col, user_id_col, date_col]).apply(
    lambda x: size_query_per_user(x['clean_query'], max_size, min_size))

In [211]:
filters_per_user_date_categ_groupby.shape, queries_per_user_date_categ_groupby.shape

((212747,), (182879,))

In [212]:
q_df_per_user_date_categ = queries_per_user_date_categ_groupby.reset_index()
q_df_per_user_date_categ.columns = ['leaf_cat', 'user_id', 'date', 'sizes']
q_df_per_user_date_categ['user_id'] = q_df_per_user_date_categ['user_id'].astype(int)
q_df_per_user_date_categ = q_df_per_user_date_categ[q_df_per_user_date_categ.sizes.astype(bool)]
q_df_per_user_date_categ.shape

(124639, 4)

In [204]:
df_per_user_date_categ = filters_per_user_date_categ_groupby.reset_index()
df_per_user_date_categ.columns = ['leaf_cat', 'user_id', 'date', 'sizes']
df_per_user_date_categ['user_id'] = df_per_user_date_categ['user_id'].astype(int)
df_per_user_date_categ = df_per_user_date_categ[df_per_user_date_categ.sizes.astype(bool)]
df_per_user_date_categ.shape

(202395, 4)

In [214]:
df = pd.concat([q_df_per_user_date_categ, df_per_user_date_categ])
df.shape

(327034, 4)

In [357]:
df_merged = df.groupby(['leaf_cat','user_id', 'date']).apply(lambda x: merge_counter_sizes(x['sizes'])).reset_index()

In [358]:
df_merged.shape

(307636, 4)

In [359]:
df_merged.columns = ['leaf_cat','user_id', 'date','sizes']

In [360]:
df_merged.head()

,leaf_cat,user_id,date,sizes
0,15709,9254,2023-04-12,{9.5: 1}
1,15709,11547,2023-04-11,{12.0: 1}
2,15709,17894,2023-04-13,{12.0: 1}
3,15709,22369,2023-04-11,{14.0: 2}
4,15709,22369,2023-04-12,"{13.0: 11, 14.0: 17, 13.5: 3, 11.0: 2, 14.5: 2..."


In [369]:
a = df_merged.sizes.iloc[4]

In [374]:
a.keys()

dict_keys([13.0, 14.0, 13.5, 11.0, 14.5, 15.0])

In [377]:
{key: old_dict[key] if lower_bound <= key <= upper_bound for key in a}

SyntaxError: invalid syntax (334484381.py, line 1)

In [352]:
def remove_keys_in_range(counter, lower_bound, upper_bound):  
    keys_to_remove = [key for key in counter if lower_bound <= counter[key] <= upper_bound]  
    for key in keys_to_remove:  
        del counter[key]  
    return counter  

In [363]:
tmp_merged = df_merged.copy()

In [364]:
w_max_size = 13
w_min_size = 4
series = tmp_merged[tmp_merged.leaf_cat == 95672]['sizes'].apply(lambda counter: remove_keys_in_range(counter,w_min_size,w_max_size))  

In [365]:
len(series), len(df_merged[df_merged.leaf_cat == 95672])

(42108, 42108)

In [366]:
df_merged[df_merged.leaf_cat == 95672]

,leaf_cat,user_id,date,sizes
265528,95672,7238,2023-04-12,"{9.0: 2, 9.5: 1}"
265529,95672,12885,2023-04-11,{}
265530,95672,22631,2023-04-12,{8.0: 2}
265531,95672,24916,2023-04-12,{7.0: 3}
265532,95672,25709,2023-04-12,{5.5: 1}
...,...,...,...,...
307631,95672,2516095886,2023-04-13,{11.0: 1}
307632,95672,2516098985,2023-04-13,{10.5: 1}
307633,95672,2516099914,2023-04-13,{5.5: 1}
307634,95672,2516100951,2023-04-13,{8.5: 3}


In [361]:
df_merged[df_merged.leaf_cat == 95672]

,leaf_cat,user_id,date,sizes
265528,95672,7238,2023-04-12,"{9.0: 2, 9.5: 1}"
265529,95672,12885,2023-04-11,{8.5: 4}
265530,95672,22631,2023-04-12,{8.0: 2}
265531,95672,24916,2023-04-12,{7.0: 3}
265532,95672,25709,2023-04-12,{5.5: 1}
...,...,...,...,...
307631,95672,2516095886,2023-04-13,{11.0: 1}
307632,95672,2516098985,2023-04-13,{10.5: 1}
307633,95672,2516099914,2023-04-13,{5.5: 1}
307634,95672,2516100951,2023-04-13,{8.5: 3}


In [346]:
series

265528    {9.0: 2, 9.5: 1}
265529                  {}
265530            {8.0: 2}
265531            {7.0: 3}
265532            {5.5: 1}
                ...       
307631           {11.0: 1}
307632           {10.5: 1}
307633            {5.5: 1}
307634            {8.5: 3}
307635            {7.0: 1}
Name: sizes, Length: 42108, dtype: object

In [345]:
np.array(z).max()

16.0

In [269]:
df_merged_for_spark = df_merged.copy()

In [270]:
df_merged_for_spark['sizes'] = [dict(i) for i in df_merged_for_spark['sizes']]

In [271]:
spark_df_merged = spark.createDataFrame(df_merged_for_spark)

In [272]:
spark_df_merged.printSchema()

root
 |-- leaf_cat: long (nullable = true)
 |-- user_id: long (nullable = true)
 |-- date: date (nullable = true)
 |-- sizes: map (nullable = true)
 |    |-- key: double
 |    |-- value: long (valueContainsNull = true)



In [273]:
spark_df_json = spark_df_merged.withColumn("sizes", F.to_json(F.col("sizes")))

In [274]:
spark_df_json.printSchema()

root
 |-- leaf_cat: long (nullable = true)
 |-- user_id: long (nullable = true)
 |-- date: date (nullable = true)
 |-- sizes: string (nullable = true)



In [308]:
!hdfs dfs -ls /user/b_selling_research/sneakers_size/temp200

Found 3 items
-rw-r--r--   3 b_selling_research hdmi-technology          0 2023-04-16 13:51 /user/b_selling_research/sneakers_size/temp200/_SUCCESS
drwxr-xr-x   - b_selling_research hdmi-technology          0 2023-04-16 13:51 /user/b_selling_research/sneakers_size/temp200/date=2023-04-11
drwxr-xr-x   - b_selling_research hdmi-technology          0 2023-04-16 13:51 /user/b_selling_research/sneakers_size/temp200/date=2023-04-12


In [311]:
spark_df_json.filter(F.col("date") < "2023-04-13").repartition(10) \
        .write.option("header",True) \
        .partitionBy("date") \
        .mode("overwrite") \
        .parquet("/user/b_selling_research/sneakers_size/temp200")

23/04/16 13:53:58 ERROR Utils: Uncaught exception in thread Thread-7
org.apache.hadoop.ipc.RemoteException(org.apache.hadoop.hdfs.server.namenode.LeaseExpiredException): No lease on /user/b_selling_research/sneakers_size/temp200/.spark-staging-0/_LOCK (inode 45473711052): File does not exist. Holder DFSClient_NONMAPREDUCE_1816607917_30 does not have any open files.
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.checkLease(FSNamesystem.java:3863)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.completeFileInternal(FSNamesystem.java:3951)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.completeFile(FSNamesystem.java:3918)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.complete(NameNodeRpcServer.java:989)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolServerSideTranslatorPB.complete(ClientNamenodeProtocolServerSideTranslatorPB.java:605)
	at org.apache.hadoop.ipc.ProtobufRpcEngine$Server$ProtoBufRpcInvoker.call(ProtobufRpcEngine.java:731)

In [314]:
spark_df_json.filter(F.col("date") == "2023-04-13").repartition(10) \
        .write.option("header",True) \
        .partitionBy("date") \
        .mode("append") \
        .parquet("/user/b_selling_research/sneakers_size/temp200")

23/04/16 14:00:16 ERROR Utils: Uncaught exception in thread Thread-7            
org.apache.hadoop.ipc.RemoteException(org.apache.hadoop.hdfs.server.namenode.LeaseExpiredException): No lease on /user/b_selling_research/sneakers_size/temp200/.spark-staging-0/_LOCK (inode 45473837518): File does not exist. Holder DFSClient_NONMAPREDUCE_1816607917_30 does not have any open files.
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.checkLease(FSNamesystem.java:3863)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.completeFileInternal(FSNamesystem.java:3951)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.completeFile(FSNamesystem.java:3918)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.complete(NameNodeRpcServer.java:989)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolServerSideTranslatorPB.complete(ClientNamenodeProtocolServerSideTranslatorPB.java:605)
	at org.apache.hadoop.ipc.ProtobufRpcEngine$Server$ProtoBufRpcInvoker.call(ProtobufRpcEngi

In [315]:
!hdfs dfs -ls /user/b_selling_research/sneakers_size/temp200/

Found 4 items
-rw-r--r--   3 b_selling_research hdmi-technology          0 2023-04-16 14:00 /user/b_selling_research/sneakers_size/temp200/_SUCCESS
drwxr-xr-x   - b_selling_research hdmi-technology          0 2023-04-16 13:54 /user/b_selling_research/sneakers_size/temp200/date=2023-04-11
drwxr-xr-x   - b_selling_research hdmi-technology          0 2023-04-16 13:54 /user/b_selling_research/sneakers_size/temp200/date=2023-04-12
drwxr-xr-x   - b_selling_research hdmi-technology          0 2023-04-16 14:00 /user/b_selling_research/sneakers_size/temp200/date=2023-04-13


In [299]:
a=spark.read.option("header",True) \
            .parquet("/user/b_selling_research/sneakers_size/temp100")

23/04/16 13:33:43 ERROR AsyncEventQueue: Listener EventLoggingListener threw an exception
java.util.ConcurrentModificationException
	at java.util.Hashtable$Enumerator.next(Hashtable.java:1387)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:424)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:420)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1429)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at scala.collection.mutable.MapLike.toSeq(MapLike.scala:75)
	at scala.collection.mutable.MapLike.toSeq$(MapLike.scala:72)
	at scala.collection.mutable.AbstractMap.toSeq(Map.scala:82)
	at org.apache.spark.scheduler.EventLoggingListener.redactPrope

In [300]:
a.printSchema()
a.show()

root
 |-- leaf_cat: long (nullable = true)
 |-- user_id: long (nullable = true)
 |-- sizes: string (nullable = true)
 |-- date: date (nullable = true)



+--------+-------+--------------------+----------+
|leaf_cat|user_id|               sizes|      date|
+--------+-------+--------------------+----------+
|   15709|3317994|          {"11.0":3}|2023-04-11|
|   15709|2578165|{"12.0":1,"12.5":...|2023-04-11|
|   15709|3097560|          {"10.5":1}|2023-04-11|
|   15709|4753469|          {"11.0":1}|2023-04-11|
|   15709|2484151|          {"16.0":1}|2023-04-11|
|   15709|5804813| {"10.5":1,"11.0":1}|2023-04-11|
|   15709|4399037|          {"12.0":1}|2023-04-11|
|   15709|2088317|          {"11.0":2}|2023-04-11|
|   15709|4310446|           {"9.0":3}|2023-04-11|
|   15709|2453550| {"11.0":1,"11.5":1}|2023-04-11|
|   15709|3739817|          {"10.0":1}|2023-04-11|
|   15709|3694587|           {"8.5":1}|2023-04-11|
|   15709|5774023|           {"8.0":1}|2023-04-11|
|   15709|5051612|           {"8.5":1}|2023-04-11|
|   15709|5013251|           {"8.5":1}|2023-04-11|
|   15709|6177410|   {"9.0":2,"9.5":3}|2023-04-11|
|   15709|3320537|           {"

In [301]:
b = a.filter(F.col("date") > "2023-04-11")

In [302]:
b.count()

23/04/16 13:34:20 ERROR AsyncEventQueue: Listener EventLoggingListener threw an exception
java.util.ConcurrentModificationException
	at java.util.Hashtable$Enumerator.next(Hashtable.java:1387)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:424)
	at scala.collection.convert.Wrappers$JPropertiesWrapper$$anon$6.next(Wrappers.scala:420)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1429)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at scala.collection.mutable.MapLike.toSeq(MapLike.scala:75)
	at scala.collection.mutable.MapLike.toSeq$(MapLike.scala:72)
	at scala.collection.mutable.AbstractMap.toSeq(Map.scala:82)
	at org.apache.spark.scheduler.EventLoggingListener.redactPrope

204624

In [303]:
a.count()

307636

In [244]:
merged_signals = pd.read_csv("/data/ebay/data/gfuchs/vlps/sneakers/size/pipeline/snapshot/processing/site_0_15709_95672/merged_signals.csv")
merged_signals.shape

(5579412, 9)

In [245]:
merged_signals.head()

,leaf_id,user_id,sizes,merged_std,merged_mean,merged_median,merged_mode,merged_maxMinDiff,merged_countOfEvidence
0,15709,39,Counter({9.5: 1}),0.000000,9.500000,9.50,9.5,0.0,1
1,15709,60,"Counter({13.0: 2, 7.0: 1, 12.5: 1, 5.0: 1, 12....",3.193961,10.416667,12.25,13.0,8.0,6
2,15709,192,Counter({8.5: 1}),0.000000,8.500000,8.50,8.5,0.0,1
3,15709,270,Counter({11.5: 1}),0.000000,11.500000,11.50,11.5,0.0,1
4,15709,376,Counter({9.5: 3}),0.000000,9.500000,9.50,9.5,0.0,3


In [111]:
df_date_agg = agg_sizes(df_per_user_date, 'sizes', col_prefix='filters')

In [112]:
df_date_agg.shape

(197863, 9)

In [113]:
df_date_agg

,user_id,date,sizes,filters_std,filters_mean,filters_median,filters_mode,filters_maxMinDiff,filters_countOfEvidence
0,7238,2023-04-12,"{9.0: 2, 9.5: 1}",0.235702,9.166667,9.0,9.0,0.5,3
1,9254,2023-04-12,{9.5: 1},0.000000,9.500000,9.5,9.5,0.0,1
2,11547,2023-04-11,{12.0: 1},0.000000,12.000000,12.0,12.0,0.0,1
3,12885,2023-04-11,{8.5: 4},0.000000,8.500000,8.5,8.5,0.0,4
5,22369,2023-04-11,{14.0: 2},0.000000,14.000000,14.0,14.0,0.0,2
...,...,...,...,...,...,...,...,...,...
207876,2516115817,2023-04-13,{11.5: 3},0.000000,11.500000,11.5,11.5,0.0,3
207877,2516115865,2023-04-13,"{11.0: 1, 12.0: 1}",0.500000,11.500000,11.5,11.0,1.0,2
207878,2516116218,2023-04-13,{13.0: 12},0.000000,13.000000,13.0,13.0,0.0,12
207879,2516116303,2023-04-13,{9.0: 1},0.000000,9.000000,9.0,9.0,0.0,1


In [69]:
df_per_user.shape

(159862, 2)

In [70]:
df_per_user = df_per_user[df_per_user.sizes.astype(bool)]
df_per_user.shape

(151241, 2)

In [72]:
df_agg = agg_sizes(df_per_user, 'sizes', col_prefix='filters')

In [73]:
df_agg.shape

(151241, 8)

In [74]:
df_agg

,user_id,sizes,filters_std,filters_mean,filters_median,filters_mode,filters_maxMinDiff,filters_countOfEvidence
0,7238,"{9.0: 2, 9.5: 1}",0.235702,9.166667,9.0,9.0,0.5,3
1,9254,{9.5: 1},0.000000,9.500000,9.5,9.5,0.0,1
2,11547,{12.0: 1},0.000000,12.000000,12.0,12.0,0.0,1
3,12885,{8.5: 4},0.000000,8.500000,8.5,8.5,0.0,4
5,22369,"{13.0: 11, 14.0: 19, 13.5: 3, 11.0: 2, 14.5: 2...",0.796134,13.565789,14.0,14.0,4.0,38
...,...,...,...,...,...,...,...,...
159857,2516115817,{11.5: 3},0.000000,11.500000,11.5,11.5,0.0,3
159858,2516115865,"{11.0: 1, 12.0: 1}",0.500000,11.500000,11.5,11.0,1.0,2
159859,2516116218,{13.0: 12},0.000000,13.000000,13.0,13.0,0.0,12
159860,2516116303,{9.0: 1},0.000000,9.000000,9.0,9.0,0.0,1
